In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [30]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.147.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [41]:
from google.cloud import aiplatform
import google.cloud.bigquery as bq
import langchain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema import format_document
from langchain_community.document_loaders import DirectoryLoader, TextLoader,BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore
from datetime import datetime
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore



PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [42]:
# Define our query
query = """
SELECT id,media_type,content,test_metadata 
FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns=["id"], page_content_columns=["content","media_type","test_metadata"]
)


/var/tmp/ipykernel_710571/1683864506.py:8: LangChainDeprecationWarning: The class `BigQueryLoader` was deprecated in LangChain 0.0.32 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import BigQueryLoader``.
  loader = BigQueryLoader(


In [52]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}


embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


### source

https://cloud.google.com/blog/products/ai-machine-learning/rag-with-bigquery-and-langchain-in-cloud

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
documents.extend(loader.load())
 
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5#,
   # separators=["\n\n"],
)
doc_splits = text_splitter.split_documents(documents)


# get current processing time to add it to metadata, datetime object containing current date and time
now = datetime.now()

# Add chunk number to metadata
chunk_idx=0
prev=doc_splits[0].metadata["id"]
for idx, split in enumerate(doc_splits):
    split.metadata["process_time"]=now
    if prev==split.metadata["id"]:
       split.metadata["chunk"] = chunk_idx      
    else:
        chunk_idx=0
        split.metadata["chunk"] = chunk_idx
        prev=split.metadata["id"]
    chunk_idx +=1
        

In [55]:

DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}



embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
) 

_=bq_store.add_documents(doc_splits)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


In [59]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore,BigQueryLoader
from datetime import datetime
import os
import logging
import json
    
def chunk_and_load():
    """
        Chunks the combination of page_content_columns from a given bigquery source string and generates
        text embeddings for them.

        Args:
            

        Returns:
             
        """
    
    project_id= os.environ.get("PROJECT_ID") 
    dataset= os.environ.get("DATASET")  
    table= os.environ.get("TABLE") 
    region= os.environ.get("REGION") 
    metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
    page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
    source_query_str= os.environ.get("SOURCE_QUERY_STR") 
    separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
    chunk_size= os.environ.get("CHUNK_SIZE")
    chunk_overlap= 0 if os.environ.get("CHUNK_OVERLAP")=="" else os.environ.get("CHUNK_OVERLAP")
 

    project_id= 'nine-quality-test' 
    dataset= 'my_langchain_dataset'
    table= 'doc_and_vectors'
    region= 'us-central1'
    metadata_columns= "id".split(",")
    page_content_columns= "content,media_type,test_metadata".split(',') 
    source_query_str= """
    SELECT id,media_type,content,test_metadata 
    FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
    """
    separators= "\n\n"
    chunk_size= 25
    chunk_overlap= 5

     # Load the data
    loader = BigQueryLoader(
        source_query_str, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )

    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)

    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")


    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()

    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    for idx, split in enumerate(doc_splits):
        split.metadata["process_time"]=now
        if prev==split.metadata["id"]:
           split.metadata["chunk"] = chunk_idx      
        else:
            chunk_idx=0
            split.metadata["chunk"] = chunk_idx
            prev=split.metadata["id"]
        chunk_idx +=1

        rows_to_insert.append(
                           {"id": split.metadata["id"], 
                               "process_time":split.metadata["process_time"].isoformat(),
                               "content":  split.page_content,
                               "chunk": split.metadata["chunk"]
                              }
                                     )
    logging.info (f"Metadata added to chunks")

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset}.{table}"

 
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("New rows have been added.")
    else:
        print(f"Encountered errors: {errors}")
    
    
    
#     logging.info (f"Bigquery store info is set -  ProjectID {project_id}, Region {region}, Dataset {dataset}, Table {table}")


#     _=bq_store.add_documents(doc_splits)
    
#     logging.info (f"Chunks and embeddings added to the store")

    return 'done'

# if __name__ == "__main__":
#     project_id= os.environ.get("PROJECT_ID") 
#     dataset= os.environ.get("DATASET")  
#     table= os.environ.get("TABLE") 
#     region= os.environ.get("REGION") 
#     metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
#     page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
#     source_query_str= os.environ.get("SOURCE_QUERY_STR") 
#     separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
#     chunk_size= os.environ.get("CHUNK_SIZE")
#     chunk_overlap= 0 if os.environ.get("CHUNK_OVERLAP")=="" else os.environ.get("CHUNK_OVERLAP")
 

#     project_id= 'nine-quality-test' 
#     dataset= 'my_langchain_dataset'
#     table= 'doc_and_vectors'
#     region= 'us-central1'
#     metadata_columns= "id".split(",")
#     page_content_columns= "content,media_type,test_metadata".split(',') 
#     source_query_str= """
#     SELECT id,media_type,content,test_metadata 
#     FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
#     """
#     separators= "\n\n"
#     chunk_size= 25
#     chunk_overlap= 5
    
    
#     message=chunk_and_load(project_id=project_id, dataset=dataset, table=table, region=region,\
#                         metadata_columns= metadata_columns, page_content_columns=page_content_columns, \
#                         source_query_str=source_query_str, separators=separators, chunk_size=chunk_size, \
#                        chunk_overlap=chunk_overlap)
#     return(message)
     

In [12]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:0000:01


In [17]:
import googleapiclient.discovery


def list_sub_directories(bucket_name, prefix):
    """Returns a list of sub-directories within the given bucket."""
    service = googleapiclient.discovery.build('storage', 'v1')

    req = service.objects().list(bucket=bucket_name, prefix=prefix, delimiter='/')
    
    res = req.execute()
    
    return res['prefixes']

# For the example (gs://abc/xyz), bucket_name is 'abc' and the prefix would be 'xyz/'
#print(list_sub_directories(bucket_name='raw_nine_files/sub_dir', prefix=''))

In [20]:
list(list_sub_directories(bucket_name=bucket, prefix=prefix))

['2023/', '2024/']

In [24]:
bucket='raw_nine_files'
prefix=''
l=[]
for main in list(list_sub_directories(bucket_name=bucket, prefix=prefix)):
    for dircty in list(list_sub_directories(bucket_name=bucket, prefix=main )):
       for sub_dircty in list(list_sub_directories(bucket_name=bucket, prefix=dircty )):
           print(sub_dircty)
        

2023/1/a/
2023/1/b/
2023/2/c/
2024/3/d/


In [416]:
def list_gcs_objects(bucket_name):
    """List all objects in a Google Cloud Storage bucket and print their metadata."""
    # Initialize a storage client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.get_bucket(bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix="vlt_video_extract")

    print(f"Objects in bucket '{bucket_name}':")
    for blob in blobs:
        print(f"Name: {blob.name}")
        print(f"Size: {blob.size} bytes")
        print(f"Content Type: {blob.content_type}")
        print(f"Media Link: {blob.public_url}")  # Get public URL if the object is publicly accessible
        print(f"Created: {blob.time_created}")
        print(f"Updated: {blob.updated}")
        print(f"Updated: {blob.self_link}")
        print(f"Updated: {blob.get}")
        #print(dir(blob))
        
        print("------")
    
 

In [20]:
get_video_duration("raw_nine_files","vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4")

5634.0

In [ ]:
https://ffcfb4b138e4ddd2456a59167c84bf4021642dff6ad6de28e57d52e-apidata.googleusercontent.com/download/storage/v1/b/raw_nine_files/o/2024%2F3%2Fd%2Fscreenshot_directrycontent.png?jk=AdvOr8tEXuKMrGk7hapfYy4ZPHk1yEDCUJKOHuCrx660bHpMtnBM9ujMQYZYlGC3uD9x2FfhMgvBZNcuZcVX6z301E7Zm2RjL3pJkSIu8paY66HCL6H6SLrZ4R1eCjkaxareY7JnsKrMof1amj819hbOcCMgyGfuRm74dLtW501kSwMiUwyd_6yYs6EeoeG38UVmQBiioRHQgCSMdDBu3Ej3hAwXvcIPMfCIhqvjQD7uD1OFecUTJx40xk_o_Yiy-uMA5XNsHRBiPtZ3KnUphFw5BIxArUX3DDMPFpEySCHtPIb67btv3H-FXW8MvV_1HBkw3V-xlDt3lKggVN4WRq3IeS6cF0xFD25jpDSYA-jBKL6f_U0wprpd3918-ldVEE77DysLlD7EnhIn_KErSpWi9qYWnsvqftvC3R-4iJUkehhUjtUiefJrA0uEJsZoSEeevzM_0jwtMY2Rb4LCzroTIYLDwRc4n45ILr13Rev_ATTsSx6ccGzxhfRClVaHrBRY7udek7EXIPJAx8m8MkyQCfKScX4GoWQqmPhn3hNd0LuW44r5xpO-ljrOzsBndeswe0gtwM53ns9ObCMznjoJo3DeGKFxKxsIUxmoSRaLP4bpvz7YgZEIqvrS8Mt1ZxHEXG98_meWlYyPd3A2ir7G7ZLQcxiIBm7-IKms2rmapPc3u5bM69KhjQNR93HYRxCXGzQnIhR8HYXzj9P1_6INOw3UVkBdIYX-jEFsNhkzHHTEIqrmDW_76xQ_wSrr_dOh3Oi8Ri12DXaXksMFBvbJhUGmgETi0QKfqd8EI-ReT3BbZVChbAIJ1RNUe-wd7guT8mKdpxTicWK0KpMnGaU3DL8tlY9GZwpFyRDmJRHwBP7iqVjU4M4jF2KyHwdL589yyxlheVn-fmItOag0vPxkysf4pX0qL52WNP3ch15tN2D8LePFp4S__rQwXD-vdsciDDXBH3DOeLeKWR4-h6lVkYxi6XtHVbLgOE26lurqe1Q0XQQcJYA1usDNdJUNYwuW1qYGiB7VH6PDYSrSHuVj2YB3bmsGxpxOf2vxrhhJZzIhKU3Yxy-BX-tXp0AK8C4oowhu66feVTx6dIl_d3xQlOinsq5kEbsxvb9aSSKPkuNkfFLTbksJhW2-QWHnv4Bwtf7uC6zMLxVfocy-iCZAMTkGa9icvrHJeT3htPvpTQYbHq_kOeTVZUj3Hw-LRcaUQnxQfOTLVWmGXBAyWhWr28l5zblWxYxGMXMP&isca=1

In [6]:
#video_url = "https://storage.cloud.google.com/raw_nine_files/vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4"
video_url="https://ffba87f3998dc58bc084d165a1de6442fbc82dae22183352ca8b1bc-apidata.googleusercontent.com/download/storage/v1/b/raw_nine_files/o/vlt_video_extract%2FMAAT%2FMAAT2024_1_A_HBB.mp4?jk=AdvOr8stvY3hRXD9psznNOnR68vfY46qQuSRwB7uO7udZdeU2LGGz-_54PmfT-yobQGxdyq1OJd1dnjB-KHuYc64Ky3hZ45dZPi3LocJ9kOpF2E7EAfzzbh1tnu855pcL63SAw-jIOe4AKQdfRQcXxO-KhuhEkUxHY9jthAzHPVv7wYH5zPsrKLuKIZzlfqJ-kh2KYLi-HlC9SDa9FtEPZbHXSuJZvlUXZK9exW_8Qnxlo5EtNLP386be6siB0rndlMgdKeHPTcO3r7RpXPlfeFm6hQuWDpe-vAW6wRz-BJzlnXOc1-cdXQ7_2Hz85JrdQ6SO-7Sf9fG7Nhk-wSLWGf_9hzaD4AaaVqhCJqmojJGQ1NID5iLry10QGX6r2Ce23Rp0P0dicWb0ePc6BygOVOPM2NZZWyHQO0OJB63aaXsz5MAFTA7dU0oQsz36-d4NzwcEiIfb4Sv86r5uowgeS_pBBOJrv0Y3NlmF1BJ_Eq5bN-q10S0wJ040NMQNs-fWfDKSSCBd-gWovKWk16r4DLSOvt10-EVfNWs8qHEnI9VJB_3E03XxhrONQxX2VqQoCnPMYcAob1lbfezAefR9dpy6xoXvcoB_GajqVoBBLlF6lk8cX69YxsL_gdSzNVJIbRMwVYfdipLKhXxiipUJM3GuB-OlD9OwKjpbYK2hGCxN0GR_pY5mrRREka8FAsxMpZvmEXEgqwYKwMPRCHmtfRj4uAyubVUPP-WCBEQQ_0r8VG1IBFqhLLrM4rqrWwxM5Vejjo6NPS6QP2Va5sOfptdcyLqTXpaNBesHkrO_hyC4ClOtwkhx1-x8rTPHat8VkUHUcPGHug2aVHp8l-zzTs-puLcF4N24hfkD06MVqbUWslkKHXTUiEbRiIOgg854ObQ976NsZI8GPeDjIIbCYzCZyDGUR9lYcJ98oQlopCEtPZsKrwUE-nV40WvQ2ABXGrW3szWrOlaOPQDwmwCTTSgWvlyxWlqI8txdIV3B1pccav3bkOiqlz7e2BcBc6rxaxu_P4Xy9_ZBGggkFO7dIiBizs0O6kkWoFib5985-bXbEvXNdeClyhuTSG9JML5-EiQqMvfm0_uW69Y9LtAUw9QwlMg6AP9nZytgw1aPBTDJlU6nZHM4ARrS7-Tv9QV3eUIGANS466_mAPP-M5wR3sWIy8fq7LhUCWmVxOqCt-iMgiqP6ZgyyRd6AgY1wAk-lshrySf1uDZV5xgeQrp8WhfNS19HNMexHSqreqPYIs&isca=1"
try:
    clip = VideoFileClip(video_url)
    print("Video Duration:", clip.duration)
except Exception as e:
    print("Error:", e)

Video Duration: 5634.0


In [ ]:
from google.cloud import storage
from google.oauth2 import service_account
from moviepy.editor import VideoFileClip
import io

 
# Initialize the storage client
client = storage.Client( )

# Set your bucket and file names
bucket_name = 'raw_nine_files'
object_name = 'vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4'

# Create a file-like object from GCS
bucket = client.bucket(bucket_name)
blob = bucket.blob(object_name)

# Download the blob as a byte stream
video_stream = io.BytesIO()
blob.download_to_file(video_stream)
video_stream.seek(0)  # Reset stream position to the beginning

# Use moviepy to get the video length from the stream
try:
    clip = VideoFileClip(video_stream)
    print("Video Duration (seconds):", clip.duration)
except Exception as e:
    print("Error:", e)

In [95]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore,BigQueryLoader
from datetime import datetime
import os
import logging
import json
    
def chunk_and_load(project_id: str= None, dataset: str= None, table: str= None, region: str =None,\
                        metadata_columns: list[str]=None, page_content_columns: list[str]= None, \
                        source_query_str: str= None, separators:  list[str]=None, chunk_size: int=None, \
                       chunk_overlap: int=0):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and generates
        text embeddings for them.

        Args:
            

        Returns:
             
        """


     # Load the data
    loader = BigQueryLoader(
        source_query_str, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )

    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)
    
   
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")


    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()

    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    for idx, split in enumerate(doc_splits):
        split.metadata["process_time"]=now
        if prev==split.metadata["id"]:
           split.metadata["chunk"] = chunk_idx      
        else:
            chunk_idx=0
            split.metadata["chunk"] = chunk_idx
            prev=split.metadata["id"]
        chunk_idx +=1

        rows_to_insert.append(
                           {"id": split.metadata["id"], 
                               "process_time":split.metadata["process_time"].isoformat(),
                               "content":  split.page_content,
                               "chunk": split.metadata["chunk"]
                              }
                                     )
    logging.info (f"Metadata added to chunks")

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset}.{table}"

 
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("New rows have been added.")
    else:
        print(f"Encountered errors: {errors}")
    
    
    
#     logging.info (f"Bigquery store info is set -  ProjectID {project_id}, Region {region}, Dataset {dataset}, Table {table}")


#     _=bq_store.add_documents(doc_splits)
    
#     logging.info (f"Chunks and embeddings added to the store")

    return 'done'

if __name__ == "__main__":
    project_id= os.environ.get("PROJECT_ID") 
    dataset= os.environ.get("DATASET")  
    table= os.environ.get("TABLE") 
    region= os.environ.get("REGION") 
    metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
    page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
    source_query_str= os.environ.get("SOURCE_QUERY_STR") 
    separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
    chunk_size= 1000 if str(os.environ.get("CHUNK_SIZE"))=="None" else int(str(os.environ.get("CHUNK_SIZE")))  
    chunk_overlap= 0 if str(os.environ.get("CHUNK_OVERLAP"))=="None" else int(str(os.environ.get("CHUNK_OVERLAP")))
 
 

    project_id= 'nine-quality-test' 
    dataset= 'my_langchain_dataset'
    table= 'doc_and_vectors'
    region= 'us-central1'
    metadata_columns= "id".split(",")
    page_content_columns= "content,media_type,test_metadata".split(',') 
    source_query_str= """
    SELECT id,media_type,content,test_metadata 
    FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
    """
    separators= "\\n\\n"
    chunk_size= 25
    chunk_overlap= 5
    
    
    message=chunk_and_load(project_id=project_id, dataset=dataset, table=table, region=region,\
                        metadata_columns= metadata_columns, page_content_columns=page_content_columns, \
                        source_query_str=source_query_str, separators=separators, chunk_size=chunk_size, \
                       chunk_overlap=chunk_overlap)
    print(message)

New rows have been added.
done


In [358]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_community import BigQueryLoader
from datetime import datetime
import logging
from google.cloud import bigquery
import os, json

def create_dataset(project_id,dataset_id,region_id):
    """
        Create a dataset
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str region_id:  name of the region under whcih the dataset should be created.               
    """
    
    client = bigquery.Client(project_id)
    # Check if the dataset exists
    try:
        dataset = client.get_dataset(f"{project_id}.{dataset_id}")  # Make an API request.
        print(f"Dataset '{dataset_id}' already exists.")
    except :
        # If the dataset does not exist, create it
        
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
        dataset.location = region_id # Set your desired location

        # Create the dataset
        dataset = client.create_dataset(dataset)  # Make an API request.
        print(f"Dataset '{dataset_id}' created successfully.")
        
 
def create_table(project_id,dataset_id,table_id):
    
    """
        Create a table 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table.
        Returns:
            list[bigquery.SchemaField] schema: table schema- list of columns
             
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("request_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("original_content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("media_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("path", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("test_metadata", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("chunk", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("process_time", "DATETIME", mode="REQUIRED"),
    ]
                

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
       
        print(f"Table '{full_table_id}' already exists.")
        
        # Truncate the table if exist
        query = f"TRUNCATE TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' truncated successfully.")
        except Exception as e:
            print(f"Error truncating table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)
        #add partitioning
        # table.time_partitioning = bigquery.TimePartitioning(
        #     type_=bigquery.TimePartitioningType.DAY,
        #     field="request_id",  # This field will be used for partitioning
        # )

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema


In [176]:
from datetime import datetime
def chunk_bq_content(request_args):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and load the result into bigquery
             
    """
    status=''
 
    try:
        project_id=  request_args['project_id']
        dataset_id=  request_args['dataset']
        table= request_args['table']
        region= request_args['region']
        metadata_columns= str(request_args['metadata_columns']).split(',') 
        page_content_columns= str(request_args['page_content_columns']).split(',') 
        source_query_str= request_args['source_query_str']
        separators= None if str(request_args['separators'])=="" else str(request_args['separators']).split(',') 
        chunk_size= 1000 if str(request_args['chunk_size']) in ["None",""] else int(str(request_args['chunk_size']))  
        chunk_overlap= 0 if str(request_args['chunk_overlap']) in ["None",""] else int(str(request_args['chunk_overlap']))  
    except:        
            project_id= 'nine-quality-test' 
            dataset_id= 'langchain_dataset'
            table= 'chunked_data'
            region= 'us-central1'
            metadata_columns= [ col.strip() for col in  "id,content,path,media_type,test_metadata".split(",")]
            page_content_columns= [col.strip() for col in "content,test_metadata".split(',') ]
            source_query_str= """
            SELECT id,media_type,content,test_metadata, path
            FROM `nine-quality-test.Nine_Quality_Test.test_long_artcles` ;
            """
            separators= "\n"
            chunk_size= 500
            chunk_overlap= 10
            #return {'record_count':0, 'status':'ERROR- Set required input parameters'}



     # Load the data
    loader = BigQueryLoader(
        query=source_query_str, project=project_id, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )
    
    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)    
     
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")

    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()
    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    request_date=datetime.today().strftime('%Y_%m_%d')
    
        
    client = bigquery.Client(project_id)    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)
    max_index=30000 #maximum number of requests in a batch
 
    max_index=2
    record_count=0
    for idx, split in enumerate(doc_splits):
            split.metadata["process_time"]=now
            if prev==split.metadata["id"]:
               split.metadata["chunk"] = chunk_idx      
            else:
                chunk_idx=0
                split.metadata["chunk"] = chunk_idx
                prev=split.metadata["id"]
            chunk_idx +=1
            version=idx // max_index
            request_id = request_date+'_'+str(version)
            rows_to_insert.append(
                               {  "request_id":  request_id  , 
                                   "id": split.metadata["id"], 
                                   "process_time":split.metadata["process_time"].isoformat(),
                                   "content":  split.page_content,
                                   "original_content": split.metadata["content"],
                                   "chunk": split.metadata["chunk"],
                                   "media_type": split.metadata["media_type"],
                                   "path": split.metadata["path"],
                                   "test_metadata": split.metadata["test_metadata"]                            

                                  }
                                         )
            
            if (idx+1) % max_index==0:
               
                #create table new if does not exist
                table=f"{dataset_id}_{request_id}"
                table_schema=create_table(project_id,dataset_id,table)
                #push the data into the table
                table_id = f"{project_id}.{dataset_id}.{table}"
                dataset  = client.dataset(dataset_id)
                table = dataset.table(table)
                job_config = bigquery.LoadJobConfig()
                job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
                job_config.schema = table_schema
                job = client.load_table_from_json(rows_to_insert, table, job_config = job_config)
                #moving to next batch
                record_count=record_count+len(rows_to_insert)
                rows_to_insert=[]
                print(job.result())
    
    return {'status':'SUCCESS', 'record_count':record_count, 'count_of_tables':version+1}

In [177]:
chunk_bq_content('')

NameError: name 'BigQueryLoader' is not defined

In [33]:
import vertexai
import datetime

from vertexai.generative_models import Part
from vertexai.preview import caching

# TODO(developer): Update and un-comment below line
# PROJECT_ID = "your-project-id"

 
system_instruction = """
You are an expert researcher. You always stick to the facts in the sources provided, and never make up new facts.
Now look at these research papers, and answer the following questions.
"""

contents = [
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2312.11805v3.pdf",
        mime_type="application/pdf",
    ),
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf",
        mime_type="application/pdf",
    ),
]

cached_content = caching.CachedContent.create(
    model_name="gemini-1.5-pro-002",
    system_instruction=system_instruction,
    contents=contents,
    ttl=datetime.timedelta(days=2),
    display_name="example-cache",
)

print(cached_content.name)
# Example response:
# 1234567890

1320504668865953792


In [32]:
datetime.timedelta(days=2)

datetime.timedelta(days=2)

In [35]:
cached_content.to_dict()

{'name': 'projects/494586852359/locations/us-central1/cachedContents/1320504668865953792',
 'model': 'projects/nine-quality-test/locations/us-central1/publishers/google/models/gemini-1.5-pro-002',
 'createTime': '2024-10-10T00:05:18.148425Z',
 'updateTime': '2024-10-10T00:05:18.148425Z',
 'expireTime': '2024-10-12T00:05:18.129147Z',
 'displayName': 'example-cache'}

In [72]:
x= data['items']
x

[{'createTime': 'llm_response.body.createTime',
  'expireTime': 'llm_response.body.expireTime',
  'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4',
  'model': 'llm_response.body.model',
  'name': 'llm_response.body.name',
  'updateTime': 'llm_response.body.updateTime',
  'usageMetadata': 'llm_response.body.usageMetadata'},
 {'createTime': 'llm_response.body.createTime',
  'expireTime': 'llm_response.body.expireTime',
  'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2.mp4',
  'model': 'llm_response.body.model',
  'name': 'llm_response.body.name',
  'updateTime': 'llm_response.body.updateTime',
  'usageMetadata': 'llm_response.body.usageMetadata'}]

In [77]:
l=list(filter(lambda element: element['gcs_uri'] == 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4', x))

In [79]:
l[0]["gcs_uri"]

'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4'

In [99]:



# Import the Google Cloud client library and JSON library
from google.cloud import storage
import json

# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket('raw_nine_files')
blob = bucket.blob('test111.json')

# Download the contents of the blob as a string and then parse it using json.loads() method
data = json.loads(blob.download_as_string(client=None))
for x in data['items']:
    print(x)
    print()
    
    
data["cache_expiry_time"]

{'createTime': 'llm_response.body.createTime', 'expireTime': 'llm_response.body.expireTime', 'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4', 'model': 'llm_response.body.model', 'name': 'llm_response.body.name', 'updateTime': 'llm_response.body.updateTime', 'usageMetadata': 'llm_response.body.usageMetadata'}

{'createTime': 'llm_response.body.createTime', 'expireTime': 'llm_response.body.expireTime', 'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2.mp4', 'model': 'llm_response.body.model', 'name': 'llm_response.body.name', 'updateTime': 'llm_response.body.updateTime', 'usageMetadata': 'llm_response.body.usageMetadata'}



'2024-10-12T23:45:42.158262Z'

In [ ]:
# Download the contents of the blob as a string and then parse it using json.loads() method
data = json.loads(blob.download_as_string(client=None))
for item in enumerate( data.items):
    print(item)

In [ ]:
         if index==0:
                            request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                            rf= open(request_file.name, "a") 
                 
                         request_list=[
                               json.dumps(
                                      {
                                        "request": {
                                            "contents": {
                                                "parts": [
                                                    {
                                                        "fileData": {
                                                            "fileUri": "gs://raw_nine_files/60MI23_33_A_HBB.mp4",
                                                            "mimeType": "video/mp4"
                                                        },
                                                        "videoMetadata": {
                                                            "endOffset": {
                                                                "nanos": 0,
                                                                "seconds": 240
                                                            },
                                                            "startOffset": {
                                                                "nanos": 0,
                                                                "seconds": 120
                                                            }
                                                        }
                                                    },
                                                    {
                                                        "text": "You are an assistant tasked with describing videos for retrieval.These descriptions will be embedded and used to retrieve the raw video.\n Chapterize the video content by grouping the video content into chapters with intervals of 120 seconds and providing a concise detail for each chapter that is well optimized for retrieval.\nIf there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\n Describe important scenes in the video concisely. If you are not sure about any info, please do not make it up. \n Only consider video from 120 seconds to 240 seconds. \nIf it is the last chapter, set the endOffset to 240 instead.\n For result, follow JSON schema.<JSONSchema>{\"description\":\"A list of chapters\",\"items\":{\"properties\":{\"Content\":{\"type\":\"string\"},\"endOffset\":{\"type\":\"integer\"},\"startOffset\":{\"type\":\"integer\"}},\"required\":[\"startOffset\",\"endOffset\",\"Content\"],\"type\":\"object\"},\"type\":\"array\"}</JSONSchema>"
                                                    }
                                                ],
                                                "role": "user"
                                            },
                                            "generation_config": {
                                                "max_output_tokens": 2048,
                                                "temperature": 0.5,
                                                "top_k": 40,
                                                "top_p": 0.8
                                            },
                                            "safety_settings": [
                                                {
                                                    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_HATE_SPEECH",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_HARASSMENT",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                    "threshold": "BLOCK_NONE"
                                                }
                                            ]
                                        }
                                    }
                               )  +"\n"
                         ]

                         rf.writelines(request_list)
                         rf.flush()
 
                         if index==(max_index-1):
                                upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                rf.close() 
                                versions.append(version)                               
                                index=0
                                version +=1
                                request_list=[]
                                rf=None

                         else:                               
                                index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   

In [46]:
import vertexai

from vertexai.preview import caching

# TODO(developer): Update and un-comment below lines
# PROJECT_ID = "your-project-id"
# cache_id = "your-cache-id"
 
# cached_content = caching.CachedContent(cached_content_name=cache_id)
# cached_content.delete()

for x in caching.CachedContent.list():
     l=x.name.split("/")[-1]
     print(l)
     print(x)
     cached_content = caching.CachedContent(cached_content_name=x)
     #cached_content.delete()

7373483305540255744
<vertexai.caching._caching.CachedContent object at 0x7fcf8c5ed3c0>: {
  "name": "projects/494586852359/locations/us-central1/cachedContents/7373483305540255744",
  "model": "projects/nine-quality-test/locations/us-central1/publishers/google/models/gemini-1.5-pro-002",
  "createTime": "2024-10-16T05:34:44.833168Z",
  "updateTime": "2024-10-16T05:34:44.833168Z",
  "expireTime": "2024-10-16T07:34:07.699361Z"
}


TypeError: expected string or bytes-like object

In [54]:
def get_video_duration(request: VideoRequest):
    """ generates video length
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int 1: as SUCCESS
          
    """
    
    fs = gcsfs.GCSFileSystem()
    # Open the file stream using gcsfs
    with fs.open(gcsuri, 'rb') as video_file:
              # Use pymediainfo to extract metadata directly from the stream
              media_info = MediaInfo.parse(video_file)
              for track in media_info.tracks:
                  if track.track_type == 'Video':
                      duration= track.duration / 1000  # Convert ms to seconds
                      print(duration)
                      break

NameError: name 'VideoRequest' is not defined

In [57]:
import functions_framework
import gcsfs
from pymediainfo import MediaInfo

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# Define the input data model
class VideoRequest(BaseModel):
    url: str  # Input: video URL

# Define the output data model
class VideoResponse(BaseModel):
    duration: float  # Output: Duration in seconds

@app.post("/get-video-duration", response_model=VideoResponse)
async def get_video_duration(request: VideoRequest):
    """
    get video duration of a given gcs url
    
    """
    
    fs = gcsfs.GCSFileSystem()
    # Open the file stream using gcsfs
    with fs.open(gcsuri, 'rb') as video_file:
              # Use pymediainfo to extract metadata directly from the stream
              media_info = MediaInfo.parse(video_file)
              for track in media_info.tracks:
                  if track.track_type == 'Video':
                      duration= track.duration / 1000  # Convert ms to seconds
                      print(duration)
                      break
 

    return VideoResponse(duration=duration)



In [163]:
import functions_framework
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json
import os

def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)                              

                      
def create_video_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None, max_request_per_file: int =None, video_metadata_file: str=""):

    """create batch request  file(s) of up to 30000 for video and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int : number of generated files
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_request_per_file

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]
    

    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(dest_bucket_name)
    blob = bucket.blob(video_metadata_file)

    # Download the contents of the blob as a string and then parse it using json.loads() method
    data = json.loads(blob.download_as_string(client=None))
    video_metadata=data['items']
    

    segments_to_process=120 #segments duration
    intervals=120#intervals
    video_start =0 #where from video to start




    for blob in blobs:                         
        if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         metadata=list(filter(lambda element: element['gcs_uri'] ==gcsuri, video_metadata))[0]
                         video_duration=metadata["videoOriginalDurationSecond"]
                         cache_id=metadata["name"]
                         prev=video_start
                         for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                                offset={'start':prev, 'end':val}
                                
                                startOffset=offset['start']
                                endOffset=offset['end']
                                if endOffset>=video_duration:
                                     endOffset=video_duration
                                print(offset)
                                prev=val 
                                segment_prompt= f"Only consider video from {startOffset} seconds to {endOffset} seconds. Ignore analyzing the rest of video.\n" 
                                if index==0:
                                    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                                    rf= open(request_file.name, "a") 
                                
                                 
                                request_list=[
                                       json.dumps(
                                              {
                                                "request": 
                                                     {
                                                     "cached_content": cache_id,
                                                      "contents":  {
                                                                        "parts": [
                                                                            {
                                                                            "fileData":  {"fileUri": gcsuri, "mimeType": mimeType},
                                                                            "videoMetadata": {
                                                                                "endOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": endOffset
                                                                                },
                                                                                "startOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": startOffset
                                                                                }
                                                                            }

                                                                            },
                                                                            {
                                                                            "text": prompt_text +"\n"+ segment_prompt
                                                                            } 

                                                                        ],
                                                                        "role": "user"
                                                                        }
                                                          , 
                                                          "generation_config": 
                                                               {"max_output_tokens": max_output_tokens, 
                                                                "temperature":temperature, 
                                                                 "top_k": top_k, 
                                                                 "top_p": top_p
                                                                }
                                                          , 
                                                          "safety_settings": 
                                                           [
                                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                             "threshold": "BLOCK_NONE"
                                                             }
                                                           , 
                                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                           "threshold": "BLOCK_NONE"
                                                           },
                                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                                           "threshold": "BLOCK_NONE"
                                                           }, 
                                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                            "threshold": "BLOCK_NONE"
                                                            }


                                                           ]
                                                     }
                                              }
                                       )  +"\n"
                                 ]

                                rf.writelines(request_list)
                                rf.flush()

                                if index==(max_index-1):
                                        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                        rf.close() 
                                        versions.append(version)                               
                                        index=0
                                        version +=1
                                        request_list=[]
                                        rf=None

                                else:                               
                                        index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

@functions_framework.http
def create_batch_request_file(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
#     request_json = request.get_json(silent=True)
#     request_args = request.args

#     dest_bucket_name =request_args['destination_bucket']
#     source_bucket_name =request_args['source_bucket']
#     source_folder_name=request_args['source_folder']
#     request_file_prefix =request_args['request_file_prefix']
#     request_file_folder =request_args['request_file_folder']
#     prompt_text= request_args['prompt_text']
#     media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]

 
#     request_content= request_args['request_content']

#     if request_args and 'temperature' in request_args:
#         temperature= request_args['temperature']
#     else:
#       temperature=0.5

#     if request_args and 'max_output_tokens' in request_args:
#        max_output_tokens= request_args['max_output_tokens'] 
#     else:
#          max_output_tokens=2048

#     if request_args and 'top_p' in request_args:
#         top_p= request_args['top_p']
#     else:
#          top_p=0.5

#     if request_args and 'top_k' in request_args:
#         top_k= request_args['top_k']
#     else:
#          top_k=50

#     if request_args and 'max_request_per_file' in request_args:
#         max_request_per_file= request_args['max_request_per_file']
#     else:
#       max_request_per_file=30000
    
    dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
    source_bucket_name ='raw_nine_files'# request_args['source_bucket']
    request_file_prefix = "video_request"#request_args['request_file_prefix']
    request_file_folder =  "video_batch_request_fldr"#request_args['request_file_folder']
    prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
    media_types= ['video/mp4']#list(request_args['media_type'])
    request_content= "video"#request_args['request_content']
    source_folder_name="vlt_video_extract/OTHERS"
    temperature=0.5
    max_output_tokens=2048
    top_p=50
    top_k=0.5
    max_request_per_file=30000
    video_metadata_file="vlt_video_metadata_fldr/vlt_video_metadata.json"

    versions=0
    if  request_content=='image':
      versions=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file
                                      )  
    if  request_content=='video':
      versions=create_video_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file,
                                     video_metadata_file=video_metadata_file
                                      ) 

    return {"status":"SUCCESS","file_count":versions}


In [103]:
create_batch_request_file('')

{'start': 0, 'end': 120}
{'start': 120, 'end': 240}
{'start': 240, 'end': 360}
{'start': 360, 'end': 480}
{'start': 480, 'end': 600}
{'start': 600, 'end': 720}
{'start': 720, 'end': 840}
{'start': 840, 'end': 960}
{'start': 960, 'end': 1080}


{'status': 'SUCCESS', 'file_count': 1}

In [105]:
import functions_framework
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json


def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)                              

def create_video_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None, max_request_per_file: int =None, video_metadata_file: str=""):

    """create batch request  file(s) of up to 30000 for video and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str source_bucket_name: name of the source  gcs bucket to read files from
            str dest_folder_name: name of the destination folder name to write files to
            str source_folder_name: name of the source folder name to read files from
            str request_file_prefix: prefix of the request file name
            list mime_types: list of accepted mime_types
            str prompt_text: prompt for Gimini
            float temperature: Gimini temprature
            float top_p: Gimini top_p
            float top_k: Gimini top_k
            int max_request_per_file: max number of requests per batch
            int max_output_tokens: Gimini max_output_tokens          
            str video_metadata_file: name of video metadata

        Returns:
            int : number of generated files
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_request_per_file

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]    

    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(dest_bucket_name)
    blob = bucket.blob(video_metadata_file)

    # Download the contents of the blob as a string and then parse it using json.loads() method
    data = json.loads(blob.download_as_string(client=None))
    video_metadata=data['items']
    

    segments_to_process=120 #segments duration
    intervals=120#intervals
    video_start =0 #where from video to start

    for blob in blobs:                         
        if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         metadata=list(filter(lambda element: element['gcs_uri'] ==gcsuri, video_metadata))[0]
                         video_duration=metadata["videoOriginalDurationSecond"]
                         cache_id=metadata["name"]

                         prev=video_start
                         for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                                offset={'start':prev, 'end':val}
                                
                                startOffset=offset['start']
                                endOffset=offset['end']
                                if endOffset>=video_duration:
                                     endOffset=video_duration
                                print(offset)
                                prev=val 
                                segment_prompt= "Only consider video from" + str(startOffset)+" seconds to "+ str(endOffset)+" seconds. Ignore analyzing the rest of video.\n" 
                                if index==0:
                                    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                                    rf= open(request_file.name, "a") 
                                
                                 
                                request_list=[
                                       json.dumps(
                                              {
                                                "request": 
                                                     {
                                                      "cached_content": cache_id,
                                                      "contents":  {
                                                                        "parts": [
                                                                            {
                                                                            "fileData":  {"fileUri": gcsuri, "mimeType": mimeType},
                                                                            "videoMetadata": {
                                                                                "endOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": endOffset
                                                                                },
                                                                                "startOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": startOffset
                                                                                }
                                                                            }

                                                                            },
                                                                            {
                                                                            "text": prompt_text +"\n"+ segment_prompt
                                                                            } 

                                                                        ],
                                                                        "role": "user"
                                                                        }
                                                          , 
                                                          "generation_config": 
                                                               {"max_output_tokens": max_output_tokens, 
                                                                "temperature":temperature, 
                                                                 "top_k": top_k, 
                                                                 "top_p": top_p
                                                                }
                                                          , 
                                                          "safety_settings": 
                                                           [
                                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                             "threshold": "BLOCK_NONE"
                                                             }
                                                           , 
                                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                           "threshold": "BLOCK_NONE"
                                                           },
                                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                                           "threshold": "BLOCK_NONE"
                                                           }, 
                                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                            "threshold": "BLOCK_NONE"
                                                            }


                                                           ]
                                                     }
                                              }
                                       )  +"\n"
                                 ]

                                rf.writelines(request_list)
                                rf.flush()

                                if index==(max_index-1):
                                        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                        rf.close() 
                                        versions.append(version)                               
                                        index=0
                                        version +=1
                                        request_list=[]
                                        rf=None

                                else:                               
                                        index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

def create_image_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None, max_request_per_file: int =None):

    """create batch request  file(s) of up to 30000 for videos and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str source_bucket_name: name of the source  gcs bucket to read files from
            str dest_folder_name: name of the destination folder name to write files to
            str source_folder_name: name of the source folder name to read files from
            str request_file_prefix: prefix of the request file name
            list mime_types: list of accepted mime_types
            str prompt_text: prompt for Gimini
            float temperature: Gimini temprature
            float top_p: Gimini top_p
            float top_k: Gimini top_k
            int max_request_per_file: max number of requests per batch
            int max_output_tokens: Gimini max_output_tokens          

        Returns:
            int : number of generated files
          
    """           
         

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_request_per_file

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]

    for blob in blobs:                         
                    if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         if index==0:
                            request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                            rf= open(request_file.name, "a") 
                 
                         request_list=[
                               json.dumps(
                                      {
                                        "request": 
                                             {
                                              "contents": 
                                                  {"parts": [{ "fileData": 
                                                                 {"fileUri": gcsuri, "mimeType": mimeType}
                                                              }, 
                                                              {"text": prompt_text
                                                              }
                                                            ]
                                                              , 
                                                    "role": "user"
                                                  }
                                                  , 
                                                  "generation_config": 
                                                       {"max_output_tokens": max_output_tokens, 
                                                        "temperature":temperature, 
                                                         "top_k": top_k, 
                                                         "top_p": top_p
                                                        }
                                                  , 
                                                  "safety_settings": 
                                                   [
                                                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                     "threshold": "BLOCK_NONE"
                                                     }
                                                   , 
                                                   {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                   "threshold": "BLOCK_NONE"
                                                   },
                                                   {"category": "HARM_CATEGORY_HARASSMENT", 
                                                   "threshold": "BLOCK_NONE"
                                                   }, 
                                                   {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                    "threshold": "BLOCK_NONE"
                                                    }


                                                   ]
                                             }
                                      }
                               )  +"\n"
                         ]

                         rf.writelines(request_list)
                         rf.flush()
 
                         if index==(max_index-1):
                                upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                rf.close() 
                                versions.append(version)                               
                                index=0
                                version +=1
                                request_list=[]
                                rf=None

                         else:                               
                                index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

@functions_framework.http
def create_batch_request_file(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    try:
        request_json = request.get_json(silent=True)
        request_args = request.args

        dest_bucket_name =request_args['destination_bucket']
        source_bucket_name =request_args['source_bucket']
        source_folder_name=request_args['source_folder']
        request_file_prefix =request_args['request_file_prefix']
        request_file_folder =request_args['request_file_folder']
        prompt_text= request_args['prompt_text']
        media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]


        request_content= request_args['request_content']

        if request_args and 'video_metadata_file' in request_args:
            video_metadata_file= request_args['video_metadata_file']
        else:
          video_metadata_file=""

        if request_args and 'temperature' in request_args:
            temperature= request_args['temperature']
        else:
          temperature=1

        if request_args and 'max_output_tokens' in request_args:
           max_output_tokens= request_args['max_output_tokens'] 
        else:
             max_output_tokens=8192

        if request_args and 'top_p' in request_args:
            top_p= request_args['top_p']
        else:
             top_p=0.95

        if request_args and 'top_k' in request_args:
            top_k= request_args['top_k']
        else:
             top_k=40

        if request_args and 'max_request_per_file' in request_args:
            max_request_per_file= request_args['max_request_per_file']
        else:
          max_request_per_file=30000
    except:
            dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
            source_bucket_name ='raw_nine_files'# request_args['source_bucket']
            request_file_prefix = "video_request"#request_args['request_file_prefix']
            request_file_folder =  "video_batch_request_fldr"#request_args['request_file_folder']
            prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
            media_types= ['video/mp4']#list(request_args['media_type'])
            request_content= "video"#request_args['request_content']
            source_folder_name="vlt_video_extract/OTHERS"
            temperature=0.5
            max_output_tokens=2048
            top_p=50
            top_k=0.5
            max_request_per_file=30000
            video_metadata_file="vlt_video_metadata_fldr/vlt_video_metadata.json"

 

    versions=0
    if  request_content=='image':
      versions=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file
                                      )  
    if  request_content=='video':
      versions=create_video_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file,
                                     video_metadata_file=video_metadata_file
                                      ) 

    return {"status":"SUCCESS","file_count":versions}


In [106]:
create_batch_request_file('')

{'start': 0, 'end': 120}
{'start': 120, 'end': 240}
{'start': 240, 'end': 360}
{'start': 360, 'end': 480}
{'start': 480, 'end': 600}
{'start': 600, 'end': 720}
{'start': 720, 'end': 840}
{'start': 840, 'end': 960}
{'start': 960, 'end': 1080}


{'status': 'SUCCESS', 'file_count': 1}

In [ ]:
contents = [
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2312.11805v3.pdf",
        mime_type="application/pdf",
    ),
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf",
        mime_type="application/pdf",
    ),
]

cached_content = caching.CachedContent.create(
    model_name="gemini-1.5-pro-002",
    system_instruction=system_instruction,
    contents=contents,
    
    display_name="example-cache",
)

print(cached_content.name)

7323099284709048320


In [1]:
import vertexai

from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview import caching

# TODO(developer): Update and un-comment below lines
# PROJECT_ID = "your-project-id"
# cache_id = "your-cache-id"

 
cached_content = caching.CachedContent(cached_content_name="7323099284709048320")

model = GenerativeModel.from_cached_content(cached_content=cached_content)

response = model.generate_content("What are the papers about?")

print(response.text)
# Example response:
# The provided text is about a new family of multimodal models called Gemini, developed by Google.
# ...

The first paper, "Gemini: A Family of Highly Capable Multimodal Models", introduces Gemini 1.0, Google's family of multimodal models. It details their architecture, training, capabilities, and responsible deployment.  Gemini 1.0 models come in different sizes (Ultra, Pro, Nano) for various applications.  The paper highlights Gemini Ultra's state-of-the-art performance on many benchmarks, exceeding human expert performance on MMLU and achieving strong results in multimodal reasoning (like MMMU) and coding tasks.  The paper emphasizes responsible AI development, outlining impact assessments, safety mitigations, and evaluations to address potential harms.

The second paper, "Gemini 1.5: Unlocking multimodal understanding across millions of tokens of context", introduces Gemini 1.5 Pro, an evolution focusing on significantly expanded context length and efficiency.  It details the model's architecture as a mixture-of-experts model and the infrastructure advancements enabling its capabilitie

In [11]:
dir

AttributeError: 'GenerativeModel' object has no attribute 'generation_config'

In [ ]:
 url: ${MM_LLM_ENDPOINT}
                                auth:
                                    type: OAuth2
                                body:
                                    "instances": [
                                                    {
                                                        "image": {
                                                         mimeType:  "${item.mime_type}",
                                                        "gcsUri": "${item.gcs_uri}"  
                                                        }

                                                     } ]
                            result: llm_response

In [29]:
MM_MODEL= "multimodalembedding@001"
MM_METHOD= "predict"
PROJECT='nine-quality-test'
LOCATION='us-central1'
MM_LLM_ENDPOINT="https://" + 'us-central1' + "-aiplatform.googleapis.com" + "/v1/projects/" + PROJECT + "/locations/" + 'us-central1' + "/publishers/google/models/" + MM_MODEL+":"+MM_METHOD
 

In [20]:
import requests

# The API endpoint
url = "https://jsonplaceholder.typicode.com/posts"

# Data to be sent
data = {
     "instances": [
                                                    {
                                                        "image": {
                                                         "mimeType":  "image/png",
                                                        "gcsUri": "gs://raw_nine_files/2023/1/a/816f62dz.png"  
                                                        }

                                                     } ]
}

# A POST request to the API
response = requests.post(MM_LLM_ENDPOINT, json=data)

# Print the response
print(response.json())

{'error': {'code': 401, 'message': 'Request is missing required authentication credential. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.', 'status': 'UNAUTHENTICATED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'CREDENTIALS_MISSING', 'domain': 'googleapis.com', 'metadata': {'method': 'google.cloud.aiplatform.v1.PredictionService.Predict', 'service': 'aiplatform.googleapis.com'}}]}}


In [25]:

from google.cloud import aiplatform
api_regional_endpoint= "us-central1-aiplatform.googleapis.com"
client_options = {"api_endpoint": api_regional_endpoint}
client = aiplatform.gapic.PredictionServiceClient(
            client_options=client_options
        )

In [30]:

instances=[
                                                    {
                                                        "image": {
                                                         "mimeType":  "image/png",
                                                        "gcsUri": "gs://raw_nine_files/2023/1/a/816f62dz.png"  
                                                        }

                                                     } ]

endpoint = (
           f"projects/{PROJECT}/locations/{LOCATION}"
           "/publishers/google/models/multimodalembedding@001"
        )
response =client.predict(endpoint=endpoint, instances=instances)
response.predictions[0].get("imageEmbedding", [])

In [61]:
content_method= "generateContent"
content_model= "gemini-1.5-pro-002"
endpoint=("v1beta1/projects/"  + PROJECT+"/locations/" + LOCATION + "/publishers/google/models/"+ content_model + ":" + content_method)
 
instances={
                                       
                                        "contents": [
                                            {
                                            "role": "user",
                                            "parts": [
                                                {
                                              
                                                
                                                "fileData": {
                                                    "mimeType": "video/mp4",
                                                    "fileUri": "gs://raw_nine_files/vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4"
                                                },

                                                "videoMetadata": {
                                                    "startOffset": {
                                                    "seconds": 300,
                                                    "nanos": 0
                                                    },
                                                    "endOffset": {
                                                    "seconds": 520,
                                                    "nanos": 0
                                                    }
                                                }
                                                },
                                                 { "text": "describe what is happening in this video in the given time frame from offset 300 second to offset 520 second in details and what people are talking about."}
                                            ]
                                            }
                                        ] ,
                                        
                                        "safetySettings": [
                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"}
                                        ],
                                        "generationConfig": {
                                            "temperature": 1,
                                            "topP": 0.5,
                                            "topK": 40,
                                           # "candidateCount": integer,
                                            "maxOutputTokens": 8192,
                                            # "presencePenalty": float,
                                            # "frequencyPenalty": float,
                                            # "stopSequences": [
                                            # string
                                            # ],
                                            # "responseMimeType": string,
                                            # "responseSchema": schema,
                                            # "seed": integer,
                                            # "responseLogprobs": boolean,
                                            # "logprobs": integer,
                                            # "audioTimestamp": boolean
                                        }#,

                                       # ,"cachedContent": "projects/494586852359/locations/us-central1/cachedContents/4906355134671355904"  
                                       
} 

from google.cloud import aiplatform
api_regional_endpoint= "us-central1-aiplatform.googleapis.com"
client_options = {"api_endpoint": api_regional_endpoint}
client = aiplatform.gapic.PredictionServiceClient(
            client_options=client_options
        )

import google.cloud.aiplatform.v1beta1  as endpoint       
#response =client.generate_content(instances)
#response =client.predict(endpoint=endpoint, instances=[instances])

In [60]:
 dir(endpoint.schema.predict.prediction_v1beta1)

['ClassificationPredictionResult',
 'ImageObjectDetectionPredictionResult',
 'ImageSegmentationPredictionResult',
 'TabularClassificationPredictionResult',
 'TabularRegressionPredictionResult',
 'TextExtractionPredictionResult',
 'TextSentimentPredictionResult',
 'TimeSeriesForecastingPredictionResult',
 'VideoActionRecognitionPredictionResult',
 'VideoClassificationPredictionResult',
 'VideoObjectTrackingPredictionResult',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'gapic_version',
 'package_version',
 'types']

In [62]:
import google.auth
import google.auth.transport.requests
creds, project = google.auth.default()

# creds.valid is False, and creds.token is None
# Need to refresh credentials to populate those

auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

In [68]:
import base64
import time
import typing
import math
import numpy as np

from google.cloud import aiplatform
from google.protobuf import struct_pb2

#libraries to generate image summaries
from vertexai.vision_models import Video
from vertexai.vision_models import VideoSegmentConfig
from vertexai.vision_models import MultiModalEmbeddingModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import Image as vision_model_Image
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part as GenerativeModelPart,
    HarmBlockThreshold,
    HarmCategory,
)
from typing import Any, Dict, List, Literal, Optional, Union

from moviepy.editor import VideoFileClip
 
import requests
from google.cloud import storage
from google.cloud.aiplatform import initializer as aiplatform_initializer
import datetime

import yaml
import re

if 1==1:
         generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
         #generation_config= GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192) 
         generation_config=GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192)#, response_mime_type='application/json',
	     # response_schema=json.loads(schema))  
         #for video, BLOCK_NONE gives error. So, have to set it to BLOCK_ONLY_HIGH
         safety_settings=  {
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    }
         stream=False
        
         video_duration=2885#math.ceil(get_video_duration(video_file))
         
         video_chapters=[]
         prev=0
         log=[]
        segments_to_process=120
         
         for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                offset={'start':prev, 'end':val}
                prev=val    
                print(offset)
                startOffset=offset['start']
                endOffset=offset['end']
                chapters=[]  
                
                if endOffset>=video_duration:
                    endOffset=video_duration
                    
                video_description_prompt=f"""You are an assistant tasked with summarizing videos for retrieval.\
                     These summaries will be embedded and used to retrieve the raw video.\
                    Chapterize the video content by grouping the video content into chapters \
                    with intervals of {intervals} seconds and providing a concise detail for each chapter that is well optimized for retrieval.\
                    If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
                    Describe important scenes in the video concisely.\
                    If you are not sure about any info, please do not make it up. \
                    Only consider video from {startOffset} seconds to {endOffset} seconds. Ignore analyzing the rest of video.\
                    If it is the last chapter, set the endOffset to {endOffset} instead.\ 
                    If a chapter includes prohibited content, set chapterSummary to "".\
                    For result, follow JSON schema.<JSONSchema>{json.dumps(schema)}</JSONSchema>"
                    """   
 
        
                contents=[parts=[GenerativeModelPart.from_uri(video_file,mime_type="video/mp4", ),
                           video_description_prompt,]        

                model_response = generative_multimodal_model.generate_content(
                                    contents,
                                    generation_config=generation_config,
                                    stream=stream,
                                    safety_settings=safety_settings, )        

                response_list = []
                    # print(model_response)

                print('Called the API, processing the result now...')
                prohibited_content=False
                    

NameError: name 'video_start' is not defined

In [71]:
help(generative_multimodal_model.generate_content)

Help on method generate_content in module vertexai.generative_models._generative_models:

generate_content(contents: Union[List[ForwardRef('Content')], List[Dict[str, Any]], str, ForwardRef('Image'), ForwardRef('Part'), List[Union[str, ForwardRef('Image'), ForwardRef('Part')]]], *, generation_config: Union[ForwardRef('GenerationConfig'), Dict[str, Any], NoneType] = None, safety_settings: Union[List[ForwardRef('SafetySetting')], Dict[google.cloud.aiplatform_v1beta1.types.content.HarmCategory, google.cloud.aiplatform_v1beta1.types.content.SafetySetting.HarmBlockThreshold], NoneType] = None, tools: Optional[List[ForwardRef('Tool')]] = None, tool_config: Optional[ForwardRef('ToolConfig')] = None, stream: bool = False) -> Union[ForwardRef('GenerationResponse'), Iterable[ForwardRef('GenerationResponse')]] method of vertexai.preview.generative_models.GenerativeModel instance
    Generates content.
    
    Args:
        contents: Contents to send to the model.
            Supports either a li

In [72]:
from oauth2client.client import GoogleCredentials

try:
    creds = GoogleCredentials.get_application_default()
except Exception as e:
    quit(e)

print("Access Token:", creds.get_access_token().access_token)

Access Token: ya29.c.c0ASRK0GazjrHy8T5oxOjgzmUAvpcPg6TrD2uCutKDaCN3dLuPEdjFLtvwDqGKVErmiHO-4iC92m7j-813QysLmaxrDYuXiRI07YF4Fb4YEJx5d8ignk4uyiWgPd7WZnO9CqjQ8HxSB7ZEqZEFuz_WmtVXeSGLyccCDAmgokqzj8CbwV5o0q2Cv2A6R9SMfoLcuYUL7HzUK5QZZLIwb14-cpwKzJzXj-l8UyxGm2Qz2hlhczUKMl--WAIkOmGODPKDCOpdSajsuGRbrKpw13MKrDuZl2HITFRE0nfF1pwClYlqTgt2H_V8OtIN8YYsvNAl-Z3f0bimuUDiA095RSnPipdSqFCN3ECwJrHa-iOmXU9SZhcvQJF7Bg_rrR0ANFRmzKCjuIDly_mHBtgqjjRVH413AshIg25-IsyVXvcg3zxUBkFyJdWI8OwZOSlt8y3Rx95FZsJ8cS7SinQOo2z5wX8Xl832FYb7Jlf1jf-jOUhgraMIk6qget7VmyiJ1me6RV82FMt2mqgx5hrSzQ0o34_03zFvwe2ZpQv0r78JBjZpg9wflsJd6ZqkQqFui8207ozoqRzStc4YU9157lzs3pagOxhV3Ju1OMp32ZROwc97hBys2tSj4ZRiFecXqFYIejMYX399dlZ-qkOrdlS0yqyk3iX3jcIOUox5-rqrOXbB8qh6hrUc4p9uv_WgvMUpYUqii85-4VmXU_czOZ7RYfYoFR6Xm73ZQrMVac3BVkSv73O5vMOrboZFazhcuzJx8c8v458i8nb336m9fdp5BXma4RpiUXVrUQcg7fi0YYJgRv6wcdd78wVBrVbxtMbr0QgZ5ItIBqprY7yv8t6Uc4kcQcMdv7MykUrI_1ObV-5Z9_QgFnt7Z9ouSBnyjyOypetS1hgunf6rU9eQY6U57f16mI-wlhqvJke__mwgkxWoyF0pml9kfrYttXyJZprSb3_wmy1pn95BW-35c

In [1]:
import functions_framework
from datetime import datetime
import logging
from google.cloud import bigquery
import os, json
from google.cloud import storage

def create_dataset(project_id,dataset_id,region_id):
    """
        Create a dataset
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str region_id:  name of the region under whcih the dataset should be created.               
    """
    
    client = bigquery.Client(project_id)
    print(project_id)
    # Check if the dataset exists
    try:
        dataset = client.get_dataset(f"{project_id}.{dataset_id}")  # Make an API request.
        print(f"Dataset '{dataset_id}' already exists.")
    except :
        # If the dataset does not exist, create it
        
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
        dataset.location = region_id # Set your desired location

        # Create the dataset
        dataset = client.create_dataset(dataset)  # Make an API request.
        print(f"Dataset '{dataset_id}' created successfully.")
        
 
def create_table(project_id,dataset_id,table_id):
    
    """
        Create a table 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table.
        Returns:
            list[bigquery.SchemaField] schema: table schema- list of columns
             
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("idx", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("mime_type", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("gcs_uri", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("media_name", "STRING", mode="REQUIRED")
       
    ]     

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
        print(f"Table '{full_table_id}' already exists.")
        
        # Drop the table if exist
        query = f"DROP TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' dropped successfully.")
        except Exception as e:
            print(f"Error dropping table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema


@functions_framework.http
def get_gcs_info(request):
    """
        loads gcs metadata info into big query table
             
    """
    status=''  
    #request_args = request.args
 

#     project_id=  request_args['project_id']
#     dataset_id=  request_args['dataset']
#     table= request_args['table']
#     region= request_args['region']
#     source_bucket= request_args['source_bucket']
#     source_folder= request_args['source_folder']
#     media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]

            
    project_id=  "nine-quality-test"
    dataset_id= 'vlt_metadata'
    table= "vlt_image_metadata"
    region= "us-central1"
    source_bucket= 'raw_nine_files'
    source_folder= "2024"
    media_types= ['image/png']
    
            

    # Initialize a storage client
    storage_client = storage.Client()
    # Get the bucket
    bucket = storage_client.get_bucket(source_bucket)
    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder)
    
    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()  
    rows_to_insert=[]        
    client = bigquery.Client(project_id)    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)


    job_list=[]
    idx=0
    for blob in blobs:
        if blob.content_type in media_types:
            rows_to_insert.append(
                                {   "idx":  idx  , 
                                    "name": blob.name, 
                                    "mime_type":blob.content_type,
                                    "gcs_uri":  "gs://"+source_bucket+"/"+blob.name,
                                    "media_name":blob.name.split("/")[-1].replace("."+blob.content_type.split("/")[-1],"")
                                    }
                                            )
        
            idx=idx+1
            
    print(rows_to_insert)
    #create table new if does not exist
    table=f"{table}" 
    table_schema=create_table(project_id,dataset_id,table)
    #push the data into the table
    table_id = f"{project_id}.{dataset_id}.{table}"
    dataset  = client.dataset(dataset_id)
    table = dataset.table(table)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    job_config.schema = table_schema
    job = client.load_table_from_json(rows_to_insert, table, job_config = job_config)                
    job_list.append(job.job_id)
  
    return {'status':'SUCCESS', 'record_count':idx,'count_of_tables':1,'table_name_prefix':table,'jobs':job_list }

In [2]:
get_gcs_info('')

nine-quality-test
Dataset 'vlt_metadata' already exists.
[{'idx': 0, 'name': '2024/3/d/screenshot_directrycontent.png', 'mime_type': 'image/png', 'gcs_uri': 'gs://raw_nine_files/2024/3/d/screenshot_directrycontent.png', 'media_name': 'screenshot_directrycontent'}]
Table 'nine-quality-test.vlt_metadata.vlt_image_metadata' created successfully.


{'status': 'SUCCESS',
 'record_count': 1,
 'count_of_tables': 1,
 'table_name_prefix': TableReference(DatasetReference('nine-quality-test', 'vlt_metadata'), 'vlt_image_metadata'),
 'jobs': ['acdf507c-f282-4d3f-bce7-89aad5c12d4d']}

In [80]:
table_schema

NameError: name 'table_schema' is not defined

In [116]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE MODEL IF NOT EXISTS `vlt_media_multimodal_embeddings_prelanding.test`
REMOTE WITH CONNECTION `us-central1.vlt_multimodal_endpoint`
OPTIONS(endpoint = 'multimodalembedding@001') ;
"""

# Execute the CREATE MODEL statement
query_job = client.query(create_model_sql)

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

Model created successfully.


SyntaxError: invalid syntax (1852933557.py, line 1)

In [40]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE OR REPLACE EXTERNAL TABLE `vlt_metadata.met_images`
WITH CONNECTION `us-central1.vlt_multimodal_endpoint`
OPTIONS
  ( object_metadata = 'SIMPLE',
    uris = ['gs://raw_nine_files/vlt_video_extract/OTHERS/*']
    )
"""
# Execute the CREATE MODEL statement
query_job = client.query(create_model_sql)

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

Model created successfully.


In [41]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE OR REPLACE TABLE `vlt_metadata.met_image_embeddings`
AS
SELECT *
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `vlt_metadata.multimodal`,
    (SELECT  * FROM `nine-quality-test.vlt_metadata.met_images` LIMIT 1000));
"""


    

# Execute the CREATE MODEL statement
query_job = client.query(create_model_sql)

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

Model created successfully.


In [32]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE OR REPLACE TABLE `vlt_metadata.met_image_embeddings1`
AS
SELECT *
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `vlt_metadata.multimodal`,
    (SELECT  * FROM `nine-quality-test.vlt_metadata.met_images` LIMIT 1000));
"""


    

# Execute the CREATE MODEL statement
query_job = client.create_job( job_config={
        "query": {
            "query": create_model_sql,
        },
        "labels": {"example-label": "example-value"},
        "maximum_bytes_billed": 10000000,
    })

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

Model created successfully.


In [ ]:
query_job.job_id

'bd806266-d3e0-4264-b7f1-737ece17ae8d'

In [89]:
batch_size = 100

from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Initialize the loop variables
offset = 0
idx=0
more_results = True
job_list=[]
row_count=1
while more_results:
    # Define your SQL CREATE MODEL statement
    generate_mm_embedding_sql = f"""CREATE OR REPLACE TABLE `vlt_metadata.met_image_embeddings_{idx}`
    AS
    SELECT *
    FROM
      ML.GENERATE_EMBEDDING(
        MODEL `vlt_metadata.multimodal`,
        (SELECT  * FROM `nine-quality-test.vlt_metadata.met_images` LIMIT {batch_size} OFFSET {offset}));
    """

    # Execute the CREATE MODEL statement
    job = client.create_job( job_config={
            "query": {
                "query": generate_mm_embedding_sql,
            },
            #"labels": {"example-label": "example-value"},
            "maximum_bytes_billed": 10000000,
        })

    # Wait for the job to complete
    job_list.append(job.job_id)
 
    # Update offset for the next batch
    offset += batch_size
    idx =idx+1
        
    # Check if there are any results
    if offset>row_count:
        more_results = False  

In [ ]:
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL `PROJECT_ID.DATASET_ID.MODEL_NAME`,
  TABLE PROJECT_ID.DATASET_ID.TABLE_NAME,
  STRUCT(TRUE AS flatten_json_output,
    START_SECOND AS start_second,
    END_SECOND AS end_second,
    INTERVAL_SECONDS AS interval_seconds)
);

In [ ]:
${query+ " SELECT *  FROM  ML.GENERATE_EMBEDDING ( MODEL" +"`"+ METADATA_DATASET+"."+MODEL_NAME+"`,"+
                                                    " ( SELECT EVM.*  FROM `"+METADATA_DATASET+"."+METADATA_TABLE+"` EVM "+
                                                       " INNER JOIN `"+METADATA_DATASET+"."+VIDEO_METADATA_TABLE+"` VM "+
                                                        " ON EVM.uri=VM.gcs_uri "+
                                                        " WHERE VW.duration>=" + string(startSegment) +                                                    
                                                        " LIMIT " +string(limit) +" OFFSET "+ string(offset) +")),"+
                                                         " STRUCT(TRUE AS flatten_json_output,"+
                                                            string(startSegment)+ "  AS start_second,"+
                                                            string(endSegment)+ " AS end_second,"+
                                                            string(INTERVALS) + " AS interval_seconds)"
                                                        }

In [ ]:
if 1==1:
    # Define your SQL CREATE MODEL statement
    generate_mm_embedding_sql = f"""CREATE OR REPLACE TABLE `vlt_media_multimodal_embeddings_prelanding.vlt_video_multimodal_embeddings_test`
    AS
    SELECT *
    FROM
      ML.GENERATE_EMBEDDING(
        MODEL `vlt_metadata.multimodal`,
        (SELECT  * FROM `nine-quality-test.vlt_metadata.met_images`
         
        
        
        LIMIT {batch_size} OFFSET {offset}));
    """

    # Execute the CREATE MODEL statement
    job = client.create_job( job_config={
            "query": {
                "query": generate_mm_embedding_sql,
            },
            #"labels": {"example-label": "example-value"},
            "maximum_bytes_billed": 10000000,
        })

    # Wait for the job to complete
    job_list.append(job.job_id)
 
    # Update offset for the next batch
    offset += batch_size
    idx =idx+1
        
    # Check if there are any results
    if offset>row_count:
        more_results = False  

In [308]:
import functions_framework

import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_community import BigQueryLoader
from datetime import datetime
import logging
from google.cloud import bigquery
import os, json


def create_log_table(project_id,dataset_id,table_id):
    
    """
        Create a log table to keep the track of chunkings 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table. 
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("request_id", "STRING", mode="REQUIRED"),
        #bigquery.SchemaField("original_content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("asset_id", "STRING", mode="REQUIRED"),
       # bigquery.SchemaField("media_type", "STRING", mode="NULLABLE"),
        #bigquery.SchemaField("path", "STRING", mode="NULLABLE"),
       # bigquery.SchemaField("test_metadata", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("chunk", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("process_time", "DATETIME", mode="REQUIRED"),
    ]
                

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
        print(f"Table '{full_table_id}' already exists.")
        
        # Drop the table if exist
        query = f"DROP TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' dropped successfully.")
            
            # recreate the table
            table = bigquery.Table(full_table_id, schema=schema)

            # Create the table
            table = client.create_table(table)  # Make an API request.
            print(f"Table '{full_table_id}' created successfully.")
        
        except Exception as e:
            print(f"Error dropping/ recreating table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema 
    
    
def create_dataset(project_id,dataset_id,region_id):
    """
        Create a dataset
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str region_id:  name of the region under whcih the dataset should be created.               
    """
    
    client = bigquery.Client(project_id)
    print(project_id)
    # Check if the dataset exists
    try:
        dataset = client.get_dataset(f"{project_id}.{dataset_id}")  # Make an API request.
        print(f"Dataset '{dataset_id}' already exists.")
    except :
        # If the dataset does not exist, create it
        
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
        dataset.location = region_id # Set your desired location

        # Create the dataset
        dataset = client.create_dataset(dataset)  # Make an API request.
        print(f"Dataset '{dataset_id}' created successfully.")
        
 
def create_table(project_id,dataset_id,table_id):
    
    """
        Create a table 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table.
        Returns:
            list[bigquery.SchemaField] schema: table schema- list of columns
             
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("request_id", "STRING", mode="REQUIRED"),
        #bigquery.SchemaField("original_content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("asset_id", "STRING", mode="REQUIRED"),
       # bigquery.SchemaField("media_type", "STRING", mode="NULLABLE"),
        #bigquery.SchemaField("path", "STRING", mode="NULLABLE"),
       # bigquery.SchemaField("test_metadata", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("chunk", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("process_time", "DATETIME", mode="REQUIRED"),
    ]
                

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
        print(f"Table '{full_table_id}' already exists.")
        
        # Drop the table if exist
        query = f"DROP TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' dropped successfully.")
            
            # recreate the table
            table = bigquery.Table(full_table_id, schema=schema)

            # Create the table
            table = client.create_table(table)  # Make an API request.
            print(f"Table '{full_table_id}' created successfully.")
        
        except Exception as e:
            print(f"Error dropping/ recreating table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema

def chunk_bq_content(request_args):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and load the result into bigquery
             
    """
    status=''  
  
   
  

        
    try:
            project_id=  request_args['project_id']
            dataset_id=  request_args['dataset']
            table= request_args['table']
            region= request_args['region']
            metadata_columns= [col.strip() for col in  str(request_args['metadata_columns']).split(',') ]
            page_content_columns= [col.strip() for col in str(request_args['page_content_columns']).split(',') ]
            source_query_str= request_args['source_query_str']
            #separators= "\n" if str(request_args['separators'])=="" else str(request_args['separators']).split(',') 
            chunk_size= 1000 if str(request_args['chunk_size']) in ["None",""] else int(str(request_args['chunk_size']))  
            chunk_overlap= 0 if str(request_args['chunk_overlap']) in ["None",""] else int(str(request_args['chunk_overlap'])) 
            max_prompt_count_limit=30000 if str(request_args['max_prompt_count_limit']) in ["None",""] else int(str(request_args['max_prompt_count_limit'])) 

    except:        
            project_id= 'nine-quality-test' 
            dataset_id= 'langchain_dataset'
            table= "chucked_content_data_2024_10_24T014648231552Z"
            region= 'us-central1'
            metadata_columns= ["asset_id"]
            page_content_columns= ["HeadLine","content"]
            source_query_str= """
           SELECT asset_id, headline as HeadLine, plain_text_column as Content  FROM `nine-quality-test.vlt_media_content_prelanding.vlt_article_content` ;
            """
   
            chunk_size= 1000
            chunk_overlap= 100
            max_prompt_count_limit=25000
            #return {'record_count':0, 'status':'ERROR- Set required input parameters'}
             

     # Load the data
    loader = BigQueryLoader(
        query=source_query_str, project=project_id, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )
    
    documents = []
    documents.extend(loader.load())
    
    print('document loaded.....')
   # print(documents)
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       #separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)    
     
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")
    
    print('document splitted.....')

    # # get current processing time to add it to metadata, datetime object containing current date and time
    # now = datetime.now()
    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["asset_id"]
    rows_to_insert=[]
    #request_date=datetime.today().strftime('%Y_%m_%d')    
        
    client = bigquery.Client(project_id)    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)
    max_index=max_prompt_count_limit #maximum number of requests in a batch
    record_count=0
    prefix=f"{table}" 
    job_list=[]
    job_execution_result={}
    print('chunk done')
    for idx, split in enumerate(doc_splits):
            split.metadata["process_time"]=now
            if prev==split.metadata["asset_id"]:
               split.metadata["chunk"] = chunk_idx  
           
            else:
                chunk_idx=0
                split.metadata["chunk"] = chunk_idx
                prev=split.metadata["asset_id"]
                      
             
                
            chunk_idx +=1
            version=idx // max_index
            request_id = prefix+'_'+str(version)
            #print(split.page_content)#.replace(page_content_columns[0]+":", "", 1).strip())
            # print(chunk_idx)
            # print()
            if chunk_idx==1:                
                 content=split.page_content  
            else:                  
                  content=page_content_columns[1]+": "+split.page_content 
                
           # print(content)
    
            rows_to_insert.append(
                               {  "request_id":  request_id  , 
                                   "asset_id": split.metadata["asset_id"], 
                                   "process_time":split.metadata["process_time"].isoformat(),
                                   "content": content,#.replace(page_content_columns[0]+":", "", 1).strip(),
                                   #"original_content": split.metadata["content"],
                                   "chunk": split.metadata["chunk"],
                                   #"media_type": split.metadata["media_type"],
                                   #"path": split.metadata["path"],
                                   #"test_metadata": split.metadata["test_metadata"]                            

                                  }
                                         )
            
            if (idx+1) % max_index==0:
                
                create_load_job()
               
                #create table new if does not exist
                table=f"{prefix}_{version}"
                print(table)
                table_schema=create_table(project_id,dataset_id,table)
                #push the data into the table
                table_id = f"{project_id}.{dataset_id}.{table}"
                dataset  = client.dataset(dataset_id)
                table = dataset.table(table)
                job_config = bigquery.LoadJobConfig()
                job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
                job_config.schema = table_schema
                job = client.load_table_from_json(rows_to_insert, table, job_config = job_config)
                #moving to next batch
                record_count=record_count+len(rows_to_insert)
                rows_to_insert=[]              
                #wait for the job to finish
                job.result()
                # Get job status 
                
                print(job.state)
                
                if job.state == 'DONE':
                    if job.error_result:
                        print(f"Job {job.job_id} for {table_id} failed with error: {job.error_result}")
                        job_execution_result['error_result']=job.error_result
                        raise Exception("Sorry, no numbers below zero")
                    else:
                        print(f"Job {job.job_id} completed successfully. For "+ table_id)
                else:
                    print(f"Job {job.job_id} for {table_id} is still in progress.")
                
                job_list.append(job_execution_result)
                
    if len(rows_to_insert)>0:
        
  
    return   {'status':'SUCCESS', 'record_count':record_count,'count_of_tables':version+1 }



if __name__ == "__main__":   

        request_args={}
    
        if  'project_id' in os.environ:
            request_args['project_id']= os.environ.get('project_id')
            
        if  'dataset' in os.environ:
            request_args['dataset']= os.environ.get('dataset')
            
        if  'table' in os.environ:
            request_args['table']= os.environ.get('table')
            
        if  'region' in os.environ:
            request_args['region']= os.environ.get('region')
            
        if  'metadata_columns' in os.environ:
            request_args['metadata_columns']= os.environ.get('metadata_columns')
            
        if  'page_content_columns' in os.environ:
            request_args['page_content_columns']= os.environ.get('page_content_columns')
            
        if  'source_query_str' in os.environ:
            request_args['source_query_str']= os.environ.get('source_query_str')
            
        if  'chunk_size' in os.environ:
            request_args['chunk_size']= os.environ.get('chunk_size')
            
        if  'chunk_overlap' in os.environ:
            request_args['chunk_overlap']= os.environ.get('chunk_overlap')
            
        if  'max_prompt_count_limit' in os.environ:
            request_args['max_prompt_count_limit']= os.environ.get('max_prompt_count_limit')
        
        chunk_bq_content(request_args)
            
          
 

document loaded.....
document splitted.....
nine-quality-test
Dataset 'langchain_dataset' already exists.
chunk done
chucked_content_data_2024_10_24T014648231552Z_0
Table 'nine-quality-test.langchain_dataset.chucked_content_data_2024_10_24T014648231552Z_0' created successfully.
DONE
Job 050f7e12-6091-4a84-a4d8-e069292d94ad completed successfully. For nine-quality-test.langchain_dataset.chucked_content_data_2024_10_24T014648231552Z_0
chucked_content_data_2024_10_24T014648231552Z_1
Table 'nine-quality-test.langchain_dataset.chucked_content_data_2024_10_24T014648231552Z_1' created successfully.
DONE
Job 90253b51-df5f-4d35-a93a-10f6d77c6925 completed successfully. For nine-quality-test.langchain_dataset.chucked_content_data_2024_10_24T014648231552Z_1


In [311]:
if 1==1:
    
    status=''  
  
   
  

        
    try:
            project_id=  request_args['project_id']
            dataset_id=  request_args['dataset']
            table= request_args['table']
            region= request_args['region']
            metadata_columns= [col.strip() for col in  str(request_args['metadata_columns']).split(',') ]
            page_content_columns= [col.strip() for col in str(request_args['page_content_columns']).split(',') ]
            source_query_str= request_args['source_query_str']
            #separators= "\n" if str(request_args['separators'])=="" else str(request_args['separators']).split(',') 
            chunk_size= 1000 if str(request_args['chunk_size']) in ["None",""] else int(str(request_args['chunk_size']))  
            chunk_overlap= 0 if str(request_args['chunk_overlap']) in ["None",""] else int(str(request_args['chunk_overlap'])) 
            max_prompt_count_limit=30000 if str(request_args['max_prompt_count_limit']) in ["None",""] else int(str(request_args['max_prompt_count_limit'])) 

    except:        
            project_id= 'nine-quality-test' 
            dataset_id= 'langchain_dataset'
            table= "chucked_content_data_2024_10_24T014648231552Z"
            region= 'us-central1'
            metadata_columns= ["asset_id"]
            page_content_columns= ["HeadLine","Content"]
            source_query_str= """
           SELECT asset_id, headline as HeadLine, plain_text_column as Content  FROM `nine-quality-test.vlt_media_content_prelanding.vlt_article_content` ;
            """
   
            chunk_size= 1000
            chunk_overlap= 100
            max_prompt_count_limit=25000
            #return {'record_count':0, 'status':'ERROR- Set required input parameters'}
             

     # Load the data
    loader = BigQueryLoader(
        query=source_query_str, project=project_id, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )
    
    documents = []
    documents.extend(loader.load())
    
    print('document loaded.....')
   # print(documents)
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       #separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)    
     
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")
    
    print('document splitted.....')

    # # get current processing time to add it to metadata, datetime object containing current date and time
    # now = datetime.now()
    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["asset_id"]
    rows_to_insert=[]
    #request_date=datetime.today().strftime('%Y_%m_%d')    
        
    client = bigquery.Client(project_id)    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)
    max_index=max_prompt_count_limit #maximum number of requests in a batch
    record_count=0
    prefix=f"{table}" 
    job_list=[]
    job_execution_result={}
    print('chunk done')
   



document loaded.....
document splitted.....
nine-quality-test
Dataset 'langchain_dataset' already exists.
chunk done


In [314]:
len(documents)

62210

In [59]:
# page_content='HeadLine: Gobsmacked by Barilaro’s behaviour? Join the club\nContent: Just what ClubsNSW needs: as it is trying to convince NSW voters that it has a good heart, a bid by John Barilaro to take over the helm (“Barilaro bids to become clubs boss”, February 3). Here is a man who achieved infamy by repeatedly demonstrating questionable judgment throughout his political life. Barilaro’s attempt to take over from Josh Landis should be consigned to the same rubbish bin that holds his bid for appointment as NSW trade commissioner to New York. Kim Woo, Mascot\n I read the article about John Barilaro’s utterly unconscionable behaviour, which effectively deprived most of the Labor electorates of their emergency bushfire funding, with smoke coming out of my ears (“Barilaro’s office in fire funding storm”, February 3). The admitted pork barrelling by this government was a vote-buying strategy. Such reprehensible action needs way more than the usual slap on the wrist, so it’ll be interesting to see what happens next. Anne Ring, Coogee\n To exacerbate people’s misery, and politicise it, is the lowest of the low. The relief recovery program is there to ensure all affected by the Black Summer fires get assistance. Barilaro shows us – again and again – how little compassion he has for the people of NSW. Daniela Catalano, Haberfield\n Sadly, the revelations show two things: first, that the egregious former minister for John Barilaro appears ready to continue to serve the people of NSW in any capacity – provided it has a six-figure salary attached. Second, that while in office he didn’t have the nous to understand how pork-barrelling works. The idea is to get people to vote for you, so it is logically directed at seats you don’t have, rather than ones you do. Unless, of course, he realised the Coalition was (is) so on the nose that it needed to shore-up those seats it already held. Or maybe both of these cases are true. Greg Oehm, Robertson\n Why is there no punishment for pork barrelling? In public life, fraudulent behaviour is punishable under law. For politicians, it’s business as usual. Forget about improved sporting facilities, safer car parks and appropriate disaster relief. So long as these actions come without consequence, nothing will change. Sam Kent, Hunters Hill\n I pondered long and hard in an attempt to predict what Barilaro might include in his job application for ClubsNSW. All I could dredge up were greyhounds, koalas, brumbies, threats to blow up the government, overseas trade appointments and, then, Thursday’s revelation of his interference in the bushfire recovery program. When will our failed politicians realise that they are unemployable? Even by ClubsNSW. Rhonda Seymour, Castle Hill\n Quite appropriate really – a man who directed desperately needed bushfire recovery funds away from ALP seats now wants to run an organisation that exists on the back of problem gamblers and illegal money laundering. Sounds like the ethics will be a perfect fit. Michael McMullan, Avoca Beach\n Is it any wonder that Barilaro denied bushfire recovery funding to the Blue Mountains? Apart from it being a Labor seat, he once referred to koalas as tree rats. The man has no love of the bush. Sally Spurr, Lane Cove\n The former deputy premier is reportedly proud of his nickname Pork Barilaro. His arrogance is breathtaking, and the revelation that the joint federal and state emergency bushfire funds were distributed on partisan grounds under his direction is contemptuous. Why am I not surprised that he has now put his hand up for the recently vacated ClubsNSW job? His intervention in the emergency bushfire recovery program needs to be immediately reviewed by ICAC. Peter Neufeld, Mosman\n I suggest that regardless of which party wins in March it considers appointing a Chief Pub Tester for major policy and decision-making. The beauty of the role is that it could be filled by 99 per cent of the population, and the only equipment the chief would require is an egg timer, a bar stool and a beer or G&T. Monique Darcy, Davidson\n The integrity and transparency of the Perrottet government can be summed up in one word: Barilaro. Michael Petras, Thornleigh\n It’s hard to decide whether I feel utter contempt for the cynical interference in disaster relief or just bewilderment at such delusional ambition? Has Barilaro considered putting his hand up for POTUS in 2024? Phil Bradshaw, Naremburn\n Good old “Pork Barilaro”, the gift that keeps giving to NSW Labor. Rob Phillips, North Epping\n I’m buying up popcorn. The Barilaro ICAC will be a corker. Dorothy Kamaker, Whale Beach\n Divisive Pell not a martyr, and no saint\n Tony Abbott called Cardinal George Pell “a saint of our times” and a victim of a modern-day crucifixion (“As Pell the man is laid to rest, Pell the martyr rises”, February 3). As I understand from my 13 years of Catholic education, Jesus was crucified for threatening and challenging the entrenched power structure and calcified ideologies and attitudes of the church hierarchy, not for propping it up. Monica Oppen, Stanmore \n Abbott’s grasp of the requirements for sainthood and of the array of great Catholics that Australia has produced helps me to understand even further his time as prime minister. Sister Susan Connelly, Lakemba \n My condolences to anyone praying for any ongoing meaningful contribution from Abbott to Australia. Ben Dryza, Newtown \n Abbott said “George Pell was the greatest man I’ve ever known”. Yet no knighthood. Mark Kilminster, Castle Hill \n “Warrior”, “soldier of truth”, “saint”, “hero”. Gandhi? Nelson Mandela? Martin Luther King? No. George Pell. Seriously? Julia Booth, Westleigh \n By far the most significant comment at the funeral was Abbott’s; that Pell should never have been investigated without a complaint; he should never have been charged without corroborating evidence, and should never have been convicted without a plausible case, as the High Court eventually realised. Laurie Le Claire, Epping \n Such delusions of grandeur. The deliberately divisive Pell was no more a saint than Abbott was a leader. Anne Garvan, Chatswood West\n The last time I checked, Saint Mary Mackillop is clearly more highly revered than a late cardinal. Saint Mary Mackillop is the greatest Catholic Australia has ever produced. Zara Tai, Minchinbury\n The question is, has Pell travelled northwards or southwards? David Boyd, Bondi Beach\n Pell incites strong opinions among his supporters and haters, even in death. Let us all move on with his passing, but may the Catholic Church move with integrity, compassion and justice. If the Church always tries to put the institution above the people, the institution, even after 2000 years, will not survive. Vincent Wong, Killara\n Dear Tony Abbott,How dare you turn a wilful blind eye to Cardinal Pell’s own wilful and well documented blind eye to the abuse of children. Sue Burton, Mosman\n The man Abbott describes as the “greatest I’ve ever known” was someone found by a royal commission to have protected paedophiles. What does this say about Abbott?\n Still, it’s no surprise to see a conga line of Liberal luminaries now waxing so lyrical about Cardinal Pell. In Pell they found a fellow right wing political operative par excellence, who brought the power and prestige of the Church to the project of reactionary conservatism. Michael Hinchey, New Lambton \n Clearly, Abbott lives in a parallel universe to most of us. Peter Kamenyitzky, Castle Hill \n Interesting to see the fawning adoration by Tony Abbott at the memorial service for George Pell. Including of course that he was a victim of a media witch hunt against him and the Catholic Church in general. George Pell presided over an administration that did little to redress the victims of sexual abuse nor did it acknowledge what future safeguards have been put in place. His administration was a failure. His only interest was protecting the Church and its money. His line at the royal commission, “It was of little interest to me”, perhaps shows the true colours of the man. Stephen Trevarrow, New Farm (Qld)\n How do we reconcile Tony Abbott’s proclamation that Cardinal George Pell is a “saint for our times” with Pell’s own statement that multiple claims of child sexual abuse perpetrated by Catholic priests under his watch were of not much interest to him? The man is hardly a martyr if the definition is “a person who voluntarily suffers death as the penalty of witnessing to and refusing to renounce a religion”. Meg Pickup, Ballina \n Not one letter writer has shown even a drop of human kindness (Letters, February 3). The man, a beacon of endeavour and achievement, was an early crusader in the fight against sexual abuse in the church, subsequently deemed inadequate, but revolutionary in its time. Pell endured horrifically biased treatment from the ABC and the Victorian police, tolerated a dud rap as a criminal, and was eventually totally vindicated, yet, rather than bitterness or hatred, he displayed only his enduring faith in his God.\n How was Pell not a giant of a man? Flawed yes, but none of us is perfect, and he didn’t ever deserve the viciousness he faced. Convicted serial killers are shown more tolerance. Rosemary O’Brien, Ashfield\n Bernini explained that since St. Peter’s is “the mother church of nearly all the others, it had to have colonnades, which would show it as if stretching out its arms maternally to receive Catholics, so as to confirm them in their faith, heretics to reunite them to the church, and infidels, to enlighten them in the true faith.” Pell’s funeral made plain for all the church in Australia has closed its arms to the wounded, those it harmed, ignored their cries of pain, and in so doing, failed to follow Jesus’ admonition to welcome and comfort the injured and hold them close, invite them in to St Mary’s Cathedral and acknowledge its crimes against the young innocents. Instead, it denied the findings of a royal commission, denied their very existence. The beginning of the road to redemption is to first admit your sins. This week, the Catholic Church did the opposite, it said nothing happened, there were no victims and proved it is too arrogant and out of touch, it can only deny its acts by washing away the blood of the young innocent. Philip Drew, Annandale\n Thank goodness for former PM John Howard, Tony Abbott, Peter Dutton, Matt Canavan, and Dan Tehan for standing up to witness the passing of Cardinal Pell.It is disappointing our other current political leaders were too small-minded, self-serving, and “go with the flow” to honour Pell who was used as a scapegoat, and suffered 404 days in gaol for a crime he did not commit. But he is in a better place now, thank God. Elizabeth Vickers, Maroubra\n \n Nevermind the monarchists, get the royals off our notes\n Does the decision to replace the late Queen’s head with an Indigenous design on a single banknote really constitute a poke in the eye to monarchists, or is it simply an attempt to acknowledge the nation’s First People on our currency (“Timing of new $5 note without King could be better”, February 3)? It seems an over reaction to a minor change that will probably go unnoticed by most of the population. After all, how many of us could accurately describe exactly what is on each banknote without looking? Merona Martin, Meroo Meadow\n I don’t understand why the dysfunctional Windsor family are on Australia’s currency anyway. James Duggan, Hunters Hill\n The monarchists seem so sad over the $5 note business. Why not cheer them up by having more royals on our money, reminding us colonials how great the monarchy has been. Put Charles on the fiver and Andrew on the $10. The $20 could have the former Edward VIII and Wallace. The $50 would be perfect for Henry VIII, circled by his five wives. Nothing less than the $100 note would do for Prince Harry. Richard Macey, Pendle Hill \n I applaud the decision not to issue a $5 note with King Charles on it. They are obviously waiting until after the coronation; any balding man looks better wearing a hat or a large crown. Noel Mills, Avalon Beach\n Better pay the best cure for GP crisis\n The government’s approach is novel: leave the cause untreated and see if the patient survives (“Doctor pay unaddressed in report”, February 3). When the main cause of the crisis in general practice is that the bulk-billing Medicare rebate is about half of where it should have been had it kept pace with inflation over the last 30 years – making bulk-billing economically unviable and leading medical graduates to pursue other paths where their expertise is more valued – the government seems to want to waste money on doing everything it can to avoid actually treating the cause. The prognosis isn’t good. Alan Garrity, North Narrabeen \n A close family friend has had his very successful inner-city GP practice up for sale for three years with no buyers and has now been forced to simply shut it down. Payments to GPs have not increased markedly for many years unlike their business overheads, resulting in marginal profitability for doctors often working in excess of 60 hours a week. What a sad commentary on our modern society when highly trained and respected GPs, whose opinions often determine the length and quality of our lives, struggle to make a living while tradies earn $150 an hour and drive around in $100,000 utes. Tony Snellgrove, Tumbulgum\n \n Inequality needs new thinking\n Decades of neo-liberal economics along with the slow burn of fiscal austerity have driven the relentless selling-off of government instrumentalities and the ransacking of many vital government departments (“Shrill critics of Chalmers’ essay are missing the point,” February 3). It provided previous governments with the revenue to win votes through tax cuts and nepotistic funding rorts. It made winners of the well-heeled, while more of us became steadily less and less well off.\n Inequality has now reached epidemic proportions and areas such as education, health, housing, aged care and transport are in fiscal crisis and desperate for funding. For decades, we have been sold the lie that taxes are a burden on our economy and to be feared. It is time we considered a more equitable and inclusive way of improving the wealth of our nation. Bruce Spence, Balmain\n We now have a treasurer who writes an essay offering new ideas to create a fairer, better society for all Australians. His ideas are progressive, thought-provoking and challenging. The voices that criticise him, immediately label him a communist and a real danger to corporate Australia. The attitude by those that have, against those that have not, are predictable, archaic, self-centred and selfish. Jim Chalmers is thinking about our future not the next election, surely that’s worth consideration. Geoff Hermon, Maraylya\n Baffling gas decision\n One of the few good decisions made by Scott Morrison is likely to be overturned (“Scott Morrison’s decision to stop controversial NSW gas field to be overturned”, smh.com.au, February 3). Whatever Morrison’s motives, this was a decision which went some way to address climate change concerns. It is now likely to be overturned with the help of a federal government that claims to care about climate change and global warming. The reason for this decision by the federal government is, apparently, the potential cost of compensation to the gas companies. Once again money trumps public interest.As a long-term Labor supporter I am baffled and deeply troubled by their approach to this issue. Edward Quinn, St Ives \n The Wounded Kangaroo\n Alan Joyce’s perpetual apologies for Qantas failings wear thin when your QF27 luggage does not arrive in Buenos Aires three days before departing an Antarctic voyage (“Despite the hype, Qantas turnbacks are a sign of strong safety”, February 3). No apologies. No explanation. And no expedition gear. Just anxiety.\n Unfortunately, Qantas no longer brings an offering to market which is as certain as it may be safe.\n It’s time for a management rethink because, right now, the Spirit of Australia days are over. Any airline will do. Stephen Goddard, Bowral\n Classroom conundrum\n Decisions are complex, not binary. Should single sex schools exist? It depends (“Enrolments at public boys’ high schools in steep decline”, February 3).\n For some children they work. For others not. It is about the makeup of the individual, not social engineering. My son went to Homebush Boys High and it worked for him. Would he have done better in a co-ed school? We will never know.\n The pendulum is swinging towards co-ed schools. I predict in a couple of decades it will start swinging the other way. Why do we think there has to be one answer? Neville Turbit, Russell Lea\n Dutton must grow up\n Ossified, obsolete and irrelevant, the Liberal Party, with or without the Duttons of this world, must soon begin to transform to 21st century life or face extinction (“Dutton must pass Harbour test”, February 3). The fresh Labor government has shaped up well so far but needs a viable and intelligent opposition for our democracy to function as we all would wish. Why is no one rising to the challenge? Brian Haisman, Winmalee\n Nick Bryant is one among many who point out the difficulties ahead for Peter Dutton as leader of the Liberal party. I wonder if a 2023 version of Don Chipp will emerge from the moderate malcontents of the party. John Bailey, Canterbury\n Does Peter Dutton know what’s really “an attack on our society”? He plays the divide and conquer card while aiming to appeal to the tone-deaf. Trying to ignite a fire when none need to exist. It’s well past time to pay our due to our First Nations on whose land we live. It’s time to grow up. Francesca Stahlut, Armidale\n Only stand up for a standing ovation\n Theatregoers standing for the curtain call ovation are a minor irritation (Letters, February 3). The real problem punters are the concertgoers who stand in their seats for half of the show. John Swanton, Coogee\n Over the decades I have attended countless music concerts and many had the audience give a standing ovation. There was one concert where the crowd got to its feet at the end not in appreciation but to demand the band return to the stage as their performance had been so short.\n When it appeared they had no intention of playing more, the audience took to throwing cushions and ripping seats out to throw at the stage in disgust. This was Elvis Costello’s first concert in Sydney in 1978 at the Regent theatre; his future ones were much longer. Con Vaitsas, Ashbury\n \n Looking for a three-headed emu in a haystack\n Good news that the radioactive pod was located in Western Australia, but I await the first appearance of a three-headed emu (Letters, February 3). Mark Berg, Caringbah South\n \n\nPut ‘woke’ back to sleep\n Could we please put “woke” back to sleep. It has become another boring, pointless word that spoils a good argument or description. Kath Maher, Lidcombe\n Postscript\n Kim Crawford of Springwood was one of a number of letter writers who claimed to have double-checked the date on Friday’s Herald front page. So confounding were the headlines - Cardinal George Pell described as a “martyr” and a “saint of our times”; the former NSW deputy premier eyeing off the newly vacated job of ClubsNSW chairman, and living up to his nickname of “pork Barilaro” - that correspondents thought they’d woken up on April 1.\n George Fishman of Vaucluse was one of only a few who wrote that “whatever side you are on, the funeral oration by Tony Abbott hailing George Pell was one for the ages”. Not surprisingly, the majority of writers were scathing of the former PM’s statement that “Pell was the greatest man I’ve ever known”, many agreeing with Peter Kamenyitzky of Castle Hill that Abbott “clearly lives in a parallel universe to most of us”, while others suggested he needs to meet more men.\n Others, like Sue Burton of Mosman, were critical of Abbott turning “a wilful blind eye to Pell’s own wilful and well documented blind eye to the abuse of children”.\n “I respectfully suggest that John Barilaro now be awarded the title of ‘the human headline’ (with apologies to Derryn Hinch),“wrote Lorraine Hickey of Green Point. “So many juicy stories, so little time.” Phil Bradshaw of Naremburn was so bewildered at the former minister’s ambition, he wondered whether “Pork Barilaro” would consider putting his hand up for POTUS in 2024. Pat Stringa, letters editor\n To submit a letter to The Sydney Morning Herald, email letters@smh.com.au. Click here for tips on how to submit letters. The Opinion newsletter is a weekly wrap of views that will challenge, champion and inform. Sign up here.\n'

page_content='HeadLine: Gobsmacked by Barilaro’s behaviour? Join the club\nContent: Just what ClubsNSW needs: as it is trying to convince NSW voters that it has a good heart, a bid by John Barilaro to take over the helm (“Barilaro bids to become clubs boss”, February 3). Here is a man who achieved infamy by repeatedly demonstrating questionable judgment throughout his political life. Barilaro’s attempt to take over from Josh Landis should be consigned to the same rubbish bin that holds his bid for appointment as NSW trade commissioner to New York. Kim Woo, Mascot\n I read the article about John Barilaro’s utterly unconscionable behaviour, which effectively deprived most of the Labor electorates of their emergency bushfire funding, with smoke coming out of my ears (“Barilaro’s office in fire funding storm”, February 3). The admitted pork barrelling by this government was a vote-buying strategy. Such reprehensible action needs way more than the usual slap on the wrist, so it’ll be interesting to see what happens next. Anne Ring, Coogee\n To exacerbate people’s misery, and politicise it, is the lowest of the low. The relief recovery program is there to ensure all affected by the Black Summer fires get assistance. Barilaro shows us – again and again – how little compassion he has for the people of NSW. Daniela Catalano, Haberfield\n Sadly, the revelations show two things: first, that the egregious former minister for John Barilaro appears ready to continue to serve the people of NSW in any capacity – provided it has a six-figure salary attached. Second, that while in office he didn’t have the nous to understand how pork-barrelling works. The idea is to get people to vote for you, so it is logically directed at seats you don’t have, rather than ones you do. Unless, of course, he realised the Coalition was (is) so on the nose that it needed to shore-up those seats it already held. Or maybe both of these cases are true. Greg Oehm, Robertson\n Why is there no punishment for pork barrelling? In public life, fraudulent behaviour is punishable under law. For politicians, it’s business as usual. Forget about improved sporting facilities, safer car parks and appropriate disaster relief. So long as these actions come without consequence, nothing will change. Sam Kent, Hunters Hill\n I pondered long and hard in an attempt to predict what Barilaro might include in his job application for ClubsNSW. All I could dredge up were greyhounds, koalas, brumbies, threats to blow up the government, overseas trade appointments and, then, Thursday’s revelation of his interference in the bushfire recovery program. When will our failed politicians realise that they are unemployable? Even by ClubsNSW. Rhonda Seymour, Castle Hill\n Quite appropriate really – a man who directed desperately needed bushfire recovery funds away from ALP seats now wants to run an organisation that exists on the back of problem gamblers and illegal money laundering. Sounds like the ethics will be a perfect fit. Michael McMullan, Avoca Beach\n Is it any wonder that Barilaro denied bushfire recovery funding to the Blue Mountains? Apart from it being a Labor seat, he once referred to koalas as tree rats. The man has no love of the bush. Sally Spurr, Lane Cove\n The former deputy premier is reportedly proud of his nickname Pork Barilaro. His arrogance is breathtaking, and the revelation that the joint federal and state emergency bushfire funds were distributed on partisan grounds under his direction is contemptuous. Why am I not surprised that he has now put his hand up for the recently vacated ClubsNSW job? His intervention in the emergency bushfire recovery program needs to be immediately reviewed by ICAC. Peter Neufeld, Mosman\n I suggest that regardless of which party wins in March it considers appointing a Chief Pub Tester for major policy and decision-making. The beauty of the role is that it could be filled by 99 per cent of the population, and the only equipment the chief would require is an egg timer, a bar stool and a beer or G&T. Monique Darcy, Davidson\n The integrity and transparency of the Perrottet government can be summed up in one word: Barilaro. Michael Petras, Thornleigh\n It’s hard to decide whether I feel utter contempt for the cynical interference in disaster relief or just bewilderment at such delusional ambition? Has Barilaro considered putting his hand up for POTUS in 2024? Phil Bradshaw, Naremburn\n Good old “Pork Barilaro”, the gift that keeps giving to NSW Labor. Rob Phillips, North Epping\n I’m buying up popcorn. The Barilaro ICAC will be a corker. Dorothy Kamaker, Whale Beach\n \n\nDivisive Pell not a martyr, and no saint\n Tony Abbott called Cardinal George Pell “a saint of our times” and a victim of a modern-day crucifixion (“As Pell the man is laid to rest, Pell the martyr rises”, February 3). As I understand from my 13 years of Catholic education, Jesus was crucified for threatening and challenging the entrenched power structure and calcified ideologies and attitudes of the church hierarchy, not for propping it up. Monica Oppen, Stanmore \n Abbott’s grasp of the requirements for sainthood and of the array of great Catholics that Australia has produced helps me to understand even further his time as prime minister. Sister Susan Connelly, Lakemba \n My condolences to anyone praying for any ongoing meaningful contribution from Abbott to Australia. Ben Dryza, Newtown \n Abbott said “George Pell was the greatest man I’ve ever known”. Yet no knighthood. Mark Kilminster, Castle Hill \n “Warrior”, “soldier of truth”, “saint”, “hero”. Gandhi? Nelson Mandela? Martin Luther King? No. George Pell. Seriously? Julia Booth, Westleigh \n By far the most significant comment at the funeral was Abbott’s; that Pell should never have been investigated without a complaint; he should never have been charged without corroborating evidence, and should never have been convicted without a plausible case, as the High Court eventually realised. Laurie Le Claire, Epping \n Such delusions of grandeur. The deliberately divisive Pell was no more a saint than Abbott was a leader. Anne Garvan, Chatswood West\n The last time I checked, Saint Mary Mackillop is clearly more highly revered than a late cardinal. Saint Mary Mackillop is the greatest Catholic Australia has ever produced. Zara Tai, Minchinbury\n The question is, has Pell travelled northwards or southwards? David Boyd, Bondi Beach\n Pell incites strong opinions among his supporters and haters, even in death. Let us all move on with his passing, but may the Catholic Church move with integrity, compassion and justice. If the Church always tries to put the institution above the people, the institution, even after 2000 years, will not survive. Vincent Wong, Killara\n Dear Tony Abbott,How dare you turn a wilful blind eye to Cardinal Pell’s own wilful and well documented blind eye to the abuse of children. Sue Burton, Mosman\n The man Abbott describes as the “greatest I’ve ever known” was someone found by a royal commission to have protected paedophiles. What does this say about Abbott?\n Still, it’s no surprise to see a conga line of Liberal luminaries now waxing so lyrical about Cardinal Pell. In Pell they found a fellow right wing political operative par excellence, who brought the power and prestige of the Church to the project of reactionary conservatism. Michael Hinchey, New Lambton \n Clearly, Abbott lives in a parallel universe to most of us. Peter Kamenyitzky, Castle Hill \n Interesting to see the fawning adoration by Tony Abbott at the memorial service for George Pell. Including of course that he was a victim of a media witch hunt against him and the Catholic Church in general. George Pell presided over an administration that did little to redress the victims of sexual abuse nor did it acknowledge what future safeguards have been put in place. His administration was a failure. His only interest was protecting the Church and its money. His line at the royal commission, “It was of little interest to me”, perhaps shows the true colours of the man. Stephen Trevarrow, New Farm (Qld)\n How do we reconcile Tony Abbott’s proclamation that Cardinal George Pell is a “saint for our times” with Pell’s own statement that multiple claims of child sexual abuse perpetrated by Catholic priests under his watch were of not much interest to him? The man is hardly a martyr if the definition is “a person who voluntarily suffers death as the penalty of witnessing to and refusing to renounce a religion”. Meg Pickup, Ballina \n Not one letter writer has shown even a drop of human kindness (Letters, February 3). The man, a beacon of endeavour and achievement, was an early crusader in the fight against sexual abuse in the church, subsequently deemed inadequate, but revolutionary in its time. Pell endured horrifically biased treatment from the ABC and the Victorian police, tolerated a dud rap as a criminal, and was eventually totally vindicated, yet, rather than bitterness or hatred, he displayed only his enduring faith in his God.\n How was Pell not a giant of a man? Flawed yes, but none of us is perfect, and he didn’t ever deserve the viciousness he faced. Convicted serial killers are shown more tolerance. Rosemary O’Brien, Ashfield\n Bernini explained that since St. Peter’s is “the mother church of nearly all the others, it had to have colonnades, which would show it as if stretching out its arms maternally to receive Catholics, so as to confirm them in their faith, heretics to reunite them to the church, and infidels, to enlighten them in the true faith.” Pell’s funeral made plain for all the church in Australia has closed its arms to the wounded, those it harmed, ignored their cries of pain, and in so doing, failed to follow Jesus’ admonition to welcome and comfort the injured and hold them close, invite them in to St Mary’s Cathedral and acknowledge its crimes against the young innocents. Instead, it denied the findings of a royal commission, denied their very existence. The beginning of the road to redemption is to first admit your sins. This week, the Catholic Church did the opposite, it said nothing happened, there were no victims and proved it is too arrogant and out of touch, it can only deny its acts by washing away the blood of the young innocent. Philip Drew, Annandale\n Thank goodness for former PM John Howard, Tony Abbott, Peter Dutton, Matt Canavan, and Dan Tehan for standing up to witness the passing of Cardinal Pell.It is disappointing our other current political leaders were too small-minded, self-serving, and “go with the flow” to honour Pell who was used as a scapegoat, and suffered 404 days in gaol for a crime he did not commit. But he is in a better place now, thank God. Elizabeth Vickers, Maroubra\n \n Nevermind the monarchists, get the royals off our notes\n Does the decision to replace the late Queen’s head with an Indigenous design on a single banknote really constitute a poke in the eye to monarchists, or is it simply an attempt to acknowledge the nation’s First People on our currency (“Timing of new $5 note without King could be better”, February 3)? It seems an over reaction to a minor change that will probably go unnoticed by most of the population. After all, how many of us could accurately describe exactly what is on each banknote without looking? Merona Martin, Meroo Meadow\n I don’t understand why the dysfunctional Windsor family are on Australia’s currency anyway. James Duggan, Hunters Hill\n The monarchists seem so sad over the $5 note business. Why not cheer them up by having more royals on our money, reminding us colonials how great the monarchy has been. Put Charles on the fiver and Andrew on the $10. The $20 could have the former Edward VIII and Wallace. The $50 would be perfect for Henry VIII, circled by his five wives. Nothing less than the $100 note would do for Prince Harry. Richard Macey, Pendle Hill \n I applaud the decision not to issue a $5 note with King Charles on it. They are obviously waiting until after the coronation; any balding man looks better wearing a hat or a large crown. Noel Mills, Avalon Beach\n Better pay the best cure for GP crisis\n The government’s approach is novel: leave the cause untreated and see if the patient survives (“Doctor pay unaddressed in report”, February 3). When the main cause of the crisis in general practice is that the bulk-billing Medicare rebate is about half of where it should have been had it kept pace with inflation over the last 30 years – making bulk-billing economically unviable and leading medical graduates to pursue other paths where their expertise is more valued – the government seems to want to waste money on doing everything it can to avoid actually treating the cause. The prognosis isn’t good. Alan Garrity, North Narrabeen \n A close family friend has had his very successful inner-city GP practice up for sale for three years with no buyers and has now been forced to simply shut it down. Payments to GPs have not increased markedly for many years unlike their business overheads, resulting in marginal profitability for doctors often working in excess of 60 hours a week. What a sad commentary on our modern society when highly trained and respected GPs, whose opinions often determine the length and quality of our lives, struggle to make a living while tradies earn $150 an hour and drive around in $100,000 utes. Tony Snellgrove, Tumbulgum\n \n Inequality needs new thinking\n Decades of neo-liberal economics along with the slow burn of fiscal austerity have driven the relentless selling-off of government instrumentalities and the ransacking of many vital government departments (“Shrill critics of Chalmers’ essay are missing the point,” February 3). It provided previous governments with the revenue to win votes through tax cuts and nepotistic funding rorts. It made winners of the well-heeled, while more of us became steadily less and less well off.\n Inequality has now reached epidemic proportions and areas such as education, health, housing, aged care and transport are in fiscal crisis and desperate for funding. For decades, we have been sold the lie that taxes are a burden on our economy and to be feared. It is time we considered a more equitable and inclusive way of improving the wealth of our nation. Bruce Spence, Balmain\n We now have a treasurer who writes an essay offering new ideas to create a fairer, better society for all Australians. His ideas are progressive, thought-provoking and challenging. The voices that criticise him, immediately label him a communist and a real danger to corporate Australia. The attitude by those that have, against those that have not, are predictable, archaic, self-centred and selfish. Jim Chalmers is thinking about our future not the next election, surely that’s worth consideration. Geoff Hermon, Maraylya\n Baffling gas decision\n One of the few good decisions made by Scott Morrison is likely to be overturned (“Scott Morrison’s decision to stop controversial NSW gas field to be overturned”, smh.com.au, February 3). Whatever Morrison’s motives, this was a decision which went some way to address climate change concerns. It is now likely to be overturned with the help of a federal government that claims to care about climate change and global warming. The reason for this decision by the federal government is, apparently, the potential cost of compensation to the gas companies. Once again money trumps public interest.As a long-term Labor supporter I am baffled and deeply troubled by their approach to this issue. Edward Quinn, St Ives \n The Wounded Kangaroo\n Alan Joyce’s perpetual apologies for Qantas failings wear thin when your QF27 luggage does not arrive in Buenos Aires three days before departing an Antarctic voyage (“Despite the hype, Qantas turnbacks are a sign of strong safety”, February 3). No apologies. No explanation. And no expedition gear. Just anxiety.\n Unfortunately, Qantas no longer brings an offering to market which is as certain as it may be safe.\n It’s time for a management rethink because, right now, the Spirit of Australia days are over. Any airline will do. Stephen Goddard, Bowral\n Classroom conundrum\n Decisions are complex, not binary. Should single sex schools exist? It depends (“Enrolments at public boys’ high schools in steep decline”, February 3).\n For some children they work. For others not. It is about the makeup of the individual, not social engineering. My son went to Homebush Boys High and it worked for him. Would he have done better in a co-ed school? We will never know.\n The pendulum is swinging towards co-ed schools. I predict in a couple of decades it will start swinging the other way. Why do we think there has to be one answer? Neville Turbit, Russell Lea\n Dutton must grow up\n Ossified, obsolete and irrelevant, the Liberal Party, with or without the Duttons of this world, must soon begin to transform to 21st century life or face extinction (“Dutton must pass Harbour test”, February 3). The fresh Labor government has shaped up well so far but needs a viable and intelligent opposition for our democracy to function as we all would wish. Why is no one rising to the challenge? Brian Haisman, Winmalee\n Nick Bryant is one among many who point out the difficulties ahead for Peter Dutton as leader of the Liberal party. I wonder if a 2023 version of Don Chipp will emerge from the moderate malcontents of the party. John Bailey, Canterbury\n Does Peter Dutton know what’s really “an attack on our society”? He plays the divide and conquer card while aiming to appeal to the tone-deaf. Trying to ignite a fire when none need to exist. It’s well past time to pay our due to our First Nations on whose land we live. It’s time to grow up. Francesca Stahlut, Armidale\n Only stand up for a standing ovation\n Theatregoers standing for the curtain call ovation are a minor irritation (Letters, February 3). The real problem punters are the concertgoers who stand in their seats for half of the show. John Swanton, Coogee\n Over the decades I have attended countless music concerts and many had the audience give a standing ovation. There was one concert where the crowd got to its feet at the end not in appreciation but to demand the band return to the stage as their performance had been so short.\n When it appeared they had no intention of playing more, the audience took to throwing cushions and ripping seats out to throw at the stage in disgust. This was Elvis Costello’s first concert in Sydney in 1978 at the Regent theatre; his future ones were much longer. Con Vaitsas, Ashbury\n \n Looking for a three-headed emu in a haystack\n Good news that the radioactive pod was located in Western Australia, but I await the first appearance of a three-headed emu (Letters, February 3). Mark Berg, Caringbah South\n Put ‘woke’ back to sleep\n Could we please put “woke” back to sleep. It has become another boring, pointless word that spoils a good argument or description. Kath Maher, Lidcombe\n Postscript\n Kim Crawford of Springwood was one of a number of letter writers who claimed to have double-checked the date on Friday’s Herald front page. So confounding were the headlines - Cardinal George Pell described as a “martyr” and a “saint of our times”; the former NSW deputy premier eyeing off the newly vacated job of ClubsNSW chairman, and living up to his nickname of “pork Barilaro” - that correspondents thought they’d woken up on April 1.\n George Fishman of Vaucluse was one of only a few who wrote that “whatever side you are on, the funeral oration by Tony Abbott hailing George Pell was one for the ages”. Not surprisingly, the majority of writers were scathing of the former PM’s statement that “Pell was the greatest man I’ve ever known”, many agreeing with Peter Kamenyitzky of Castle Hill that Abbott “clearly lives in a parallel universe to most of us”, while others suggested he needs to meet more men.\n Others, like Sue Burton of Mosman, were critical of Abbott turning “a wilful blind eye to Pell’s own wilful and well documented blind eye to the abuse of children”.\n “I respectfully suggest that John Barilaro now be awarded the title of ‘the human headline’ (with apologies to Derryn Hinch),“wrote Lorraine Hickey of Green Point. “So many juicy stories, so little time.” Phil Bradshaw of Naremburn was so bewildered at the former minister’s ambition, he wondered whether “Pork Barilaro” would consider putting his hand up for POTUS in 2024. Pat Stringa, letters editor\n To submit a letter to The Sydney Morning Herald, email letters@smh.com.au. Click here for tips on how to submit letters. The Opinion newsletter is a weekly wrap of views that will challenge, champion and inform. Sign up here.\n'
page_content='HeadLine: Gobsmacked by Barilaro’s behaviour? Join the club\nContent: Just what ClubsNSW needs: as it is trying to convince NSW voters that it has a good heart, a bid by John Barilaro to take over the helm (“Barilaro bids to become clubs boss”, February 3). Here is a man who achieved infamy by repeatedly demonstrating questionable judgment throughout his political life. Barilaro’s attempt to take over from Josh Landis should be consigned to the same rubbish bin that holds his bid for appointment as NSW trade commissioner to New York. Kim Woo, Mascot\n I read the article about John Barilaro’s utterly unconscionable behaviour, which effectively deprived most of the Labor electorates of their emergency bushfire funding, with smoke coming out of my ears (“Barilaro’s office in fire funding storm”, February 3). The admitted pork barrelling by this government was a vote-buying strategy. Such reprehensible action needs way more than the usual slap on the wrist, so it’ll be interesting to see what happens next. Anne Ring, Coogee\n To exacerbate people’s misery, and politicise it, is the lowest of the low. The relief recovery program is there to ensure all affected by the Black Summer fires get assistance. Barilaro shows us – again and again – how little compassion he has for the people of NSW. Daniela Catalano, Haberfield\n Sadly, the revelations show two things: first, that the egregious former minister for John Barilaro appears ready to continue to serve the people of NSW in any capacity – provided it has a six-figure salary attached. Second, that while in office he didn’t have the nous to understand how pork-barrelling works. The idea is to get people to vote for you, so it is logically directed at seats you don’t have, rather than ones you do. Unless, of course, he realised the Coalition was (is) so on the nose that it needed to shore-up those seats it already held. Or maybe both of these cases are true. Greg Oehm, Robertson\n Why is there no punishment for pork barrelling? In public life, fraudulent behaviour is punishable under law. For politicians, it’s business as usual. Forget about improved sporting facilities, safer car parks and appropriate disaster relief. So long as these actions come without consequence, nothing will change. Sam Kent, Hunters Hill\n I pondered long and hard in an attempt to predict what Barilaro might include in his job application for ClubsNSW. All I could dredge up were greyhounds, koalas, brumbies, threats to blow up the government, overseas trade appointments and, then, Thursday’s revelation of his interference in the bushfire recovery program. When will our failed politicians realise that they are unemployable? Even by ClubsNSW. Rhonda Seymour, Castle Hill\n Quite appropriate really – a man who directed desperately needed bushfire recovery funds away from ALP seats now wants to run an organisation that exists on the back of problem gamblers and illegal money laundering. Sounds like the ethics will be a perfect fit. Michael McMullan, Avoca Beach\n Is it any wonder that Barilaro denied bushfire recovery funding to the Blue Mountains? Apart from it being a Labor seat, he once referred to koalas as tree rats. The man has no love of the bush. Sally Spurr, Lane Cove\n The former deputy premier is reportedly proud of his nickname Pork Barilaro. His arrogance is breathtaking, and the revelation that the joint federal and state emergency bushfire funds were distributed on partisan grounds under his direction is contemptuous. Why am I not surprised that he has now put his hand up for the recently vacated ClubsNSW job? His intervention in the emergency bushfire recovery program needs to be immediately reviewed by ICAC. Peter Neufeld, Mosman\n I suggest that regardless of which party wins in March it considers appointing a Chief Pub Tester for major policy and decision-making. The beauty of the role is that it could be filled by 99 per cent of the population, and the only equipment the chief would require is an egg timer, a bar stool and a beer or G&T. Monique Darcy, Davidson\n The integrity and transparency of the Perrottet government can be summed up in one word: Barilaro. Michael Petras, Thornleigh\n It’s hard to decide whether I feel utter contempt for the cynical interference in disaster relief or just bewilderment at such delusional ambition? Has Barilaro considered putting his hand up for POTUS in 2024? Phil Bradshaw, Naremburn\n Good old “Pork Barilaro”, the gift that keeps giving to NSW Labor. Rob Phillips, North Epping\n I’m buying up popcorn. The Barilaro ICAC will be a corker. Dorothy Kamaker, Whale Beach\n \n\nDivisive Pell not a martyr, and no saint\n Tony Abbott called Cardinal George Pell “a saint of our times” and a victim of a modern-day crucifixion (“As Pell the man is laid to rest, Pell the martyr rises”, February 3). As I understand from my 13 years of Catholic education, Jesus was crucified for threatening and challenging the entrenched power structure and calcified ideologies and attitudes of the church hierarchy, not for propping it up. Monica Oppen, Stanmore \n Abbott’s grasp of the requirements for sainthood and of the array of great Catholics that Australia has produced helps me to understand even further his time as prime minister. Sister Susan Connelly, Lakemba \n My condolences to anyone praying for any ongoing meaningful contribution from Abbott to Australia. Ben Dryza, Newtown \n Abbott said “George Pell was the greatest man I’ve ever known”. Yet no knighthood. Mark Kilminster, Castle Hill \n “Warrior”, “soldier of truth”, “saint”, “hero”. Gandhi? Nelson Mandela? Martin Luther King? No. George Pell. Seriously? Julia Booth, Westleigh \n By far the most significant comment at the funeral was Abbott’s; that Pell should never have been investigated without a complaint; he should never have been charged without corroborating evidence, and should never have been convicted without a plausible case, as the High Court eventually realised. Laurie Le Claire, Epping \n Such delusions of grandeur. The deliberately divisive Pell was no more a saint than Abbott was a leader. Anne Garvan, Chatswood West\n The last time I checked, Saint Mary Mackillop is clearly more highly revered than a late cardinal. Saint Mary Mackillop is the greatest Catholic Australia has ever produced. Zara Tai, Minchinbury\n The question is, has Pell travelled northwards or southwards? David Boyd, Bondi Beach\n Pell incites strong opinions among his supporters and haters, even in death. Let us all move on with his passing, but may the Catholic Church move with integrity, compassion and justice. If the Church always tries to put the institution above the people, the institution, even after 2000 years, will not survive. Vincent Wong, Killara\n Dear Tony Abbott,How dare you turn a wilful blind eye to Cardinal Pell’s own wilful and well documented blind eye to the abuse of children. Sue Burton, Mosman\n The man Abbott describes as the “greatest I’ve ever known” was someone found by a royal commission to have protected paedophiles. What does this say about Abbott?\n Still, it’s no surprise to see a conga line of Liberal luminaries now waxing so lyrical about Cardinal Pell. In Pell they found a fellow right wing political operative par excellence, who brought the power and prestige of the Church to the project of reactionary conservatism. Michael Hinchey, New Lambton \n Clearly, Abbott lives in a parallel universe to most of us. Peter Kamenyitzky, Castle Hill \n Interesting to see the fawning adoration by Tony Abbott at the memorial service for George Pell. Including of course that he was a victim of a media witch hunt against him and the Catholic Church in general. George Pell presided over an administration that did little to redress the victims of sexual abuse nor did it acknowledge what future safeguards have been put in place. His administration was a failure. His only interest was protecting the Church and its money. His line at the royal commission, “It was of little interest to me”, perhaps shows the true colours of the man. Stephen Trevarrow, New Farm (Qld)\n How do we reconcile Tony Abbott’s proclamation that Cardinal George Pell is a “saint for our times” with Pell’s own statement that multiple claims of child sexual abuse perpetrated by Catholic priests under his watch were of not much interest to him? The man is hardly a martyr if the definition is “a person who voluntarily suffers death as the penalty of witnessing to and refusing to renounce a religion”. Meg Pickup, Ballina \n Not one letter writer has shown even a drop of human kindness (Letters, February 3). The man, a beacon of endeavour and achievement, was an early crusader in the fight against sexual abuse in the church, subsequently deemed inadequate, but revolutionary in its time. Pell endured horrifically biased treatment from the ABC and the Victorian police, tolerated a dud rap as a criminal, and was eventually totally vindicated, yet, rather than bitterness or hatred, he displayed only his enduring faith in his God.\n How was Pell not a giant of a man? Flawed yes, but none of us is perfect, and he didn’t ever deserve the viciousness he faced. Convicted serial killers are shown more tolerance. Rosemary O’Brien, Ashfield\n Bernini explained that since St. Peter’s is “the mother church of nearly all the others, it had to have colonnades, which would show it as if stretching out its arms maternally to receive Catholics, so as to confirm them in their faith, heretics to reunite them to the church, and infidels, to enlighten them in the true faith.” Pell’s funeral made plain for all the church in Australia has closed its arms to the wounded, those it harmed, ignored their cries of pain, and in so doing, failed to follow Jesus’ admonition to welcome and comfort the injured and hold them close, invite them in to St Mary’s Cathedral and acknowledge its crimes against the young innocents. Instead, it denied the findings of a royal commission, denied their very existence. The beginning of the road to redemption is to first admit your sins. This week, the Catholic Church did the opposite, it said nothing happened, there were no victims and proved it is too arrogant and out of touch, it can only deny its acts by washing away the blood of the young innocent. Philip Drew, Annandale\n Thank goodness for former PM John Howard, Tony Abbott, Peter Dutton, Matt Canavan, and Dan Tehan for standing up to witness the passing of Cardinal Pell.It is disappointing our other current political leaders were too small-minded, self-serving, and “go with the flow” to honour Pell who was used as a scapegoat, and suffered 404 days in gaol for a crime he did not commit. But he is in a better place now, thank God. Elizabeth Vickers, Maroubra\n \n\nNevermind the monarchists, get the royals off our notes\n Does the decision to replace the late Queen’s head with an Indigenous design on a single banknote really constitute a poke in the eye to monarchists, or is it simply an attempt to acknowledge the nation’s First People on our currency (“Timing of new $5 note without King could be better”, February 3)? It seems an over reaction to a minor change that will probably go unnoticed by most of the population. After all, how many of us could accurately describe exactly what is on each banknote without looking? Merona Martin, Meroo Meadow\n I don’t understand why the dysfunctional Windsor family are on Australia’s currency anyway. James Duggan, Hunters Hill\n The monarchists seem so sad over the $5 note business. Why not cheer them up by having more royals on our money, reminding us colonials how great the monarchy has been. Put Charles on the fiver and Andrew on the $10. The $20 could have the former Edward VIII and Wallace. The $50 would be perfect for Henry VIII, circled by his five wives. Nothing less than the $100 note would do for Prince Harry. Richard Macey, Pendle Hill \n I applaud the decision not to issue a $5 note with King Charles on it. They are obviously waiting until after the coronation; any balding man looks better wearing a hat or a large crown. Noel Mills, Avalon Beach\n \n\nBetter pay the best cure for GP crisis\n The government’s approach is novel: leave the cause untreated and see if the patient survives (“Doctor pay unaddressed in report”, February 3). When the main cause of the crisis in general practice is that the bulk-billing Medicare rebate is about half of where it should have been had it kept pace with inflation over the last 30 years – making bulk-billing economically unviable and leading medical graduates to pursue other paths where their expertise is more valued – the government seems to want to waste money on doing everything it can to avoid actually treating the cause. The prognosis isn’t good. Alan Garrity, North Narrabeen \n A close family friend has had his very successful inner-city GP practice up for sale for three years with no buyers and has now been forced to simply shut it down. Payments to GPs have not increased markedly for many years unlike their business overheads, resulting in marginal profitability for doctors often working in excess of 60 hours a week. What a sad commentary on our modern society when highly trained and respected GPs, whose opinions often determine the length and quality of our lives, struggle to make a living while tradies earn $150 an hour and drive around in $100,000 utes. Tony Snellgrove, Tumbulgum\n \n\nInequality needs new thinking\n Decades of neo-liberal economics along with the slow burn of fiscal austerity have driven the relentless selling-off of government instrumentalities and the ransacking of many vital government departments (“Shrill critics of Chalmers’ essay are missing the point,” February 3). It provided previous governments with the revenue to win votes through tax cuts and nepotistic funding rorts. It made winners of the well-heeled, while more of us became steadily less and less well off.\n Inequality has now reached epidemic proportions and areas such as education, health, housing, aged care and transport are in fiscal crisis and desperate for funding. For decades, we have been sold the lie that taxes are a burden on our economy and to be feared. It is time we considered a more equitable and inclusive way of improving the wealth of our nation. Bruce Spence, Balmain\n We now have a treasurer who writes an essay offering new ideas to create a fairer, better society for all Australians. His ideas are progressive, thought-provoking and challenging. The voices that criticise him, immediately label him a communist and a real danger to corporate Australia. The attitude by those that have, against those that have not, are predictable, archaic, self-centred and selfish. Jim Chalmers is thinking about our future not the next election, surely that’s worth consideration. Geoff Hermon, Maraylya\n \n\nBaffling gas decision\n One of the few good decisions made by Scott Morrison is likely to be overturned (“Scott Morrison’s decision to stop controversial NSW gas field to be overturned”, smh.com.au, February 3). Whatever Morrison’s motives, this was a decision which went some way to address climate change concerns. It is now likely to be overturned with the help of a federal government that claims to care about climate change and global warming. The reason for this decision by the federal government is, apparently, the potential cost of compensation to the gas companies. Once again money trumps public interest.As a long-term Labor supporter I am baffled and deeply troubled by their approach to this issue. Edward Quinn, St Ives \n \n\nThe Wounded Kangaroo\n Alan Joyce’s perpetual apologies for Qantas failings wear thin when your QF27 luggage does not arrive in Buenos Aires three days before departing an Antarctic voyage (“Despite the hype, Qantas turnbacks are a sign of strong safety”, February 3). No apologies. No explanation. And no expedition gear. Just anxiety.\n Unfortunately, Qantas no longer brings an offering to market which is as certain as it may be safe.\n It’s time for a management rethink because, right now, the Spirit of Australia days are over. Any airline will do. Stephen Goddard, Bowral\n \n\nClassroom conundrum\n Decisions are complex, not binary. Should single sex schools exist? It depends (“Enrolments at public boys’ high schools in steep decline”, February 3).\n For some children they work. For others not. It is about the makeup of the individual, not social engineering. My son went to Homebush Boys High and it worked for him. Would he have done better in a co-ed school? We will never know.\n The pendulum is swinging towards co-ed schools. I predict in a couple of decades it will start swinging the other way. Why do we think there has to be one answer? Neville Turbit, Russell Lea\n \n\nDutton must grow up\n Ossified, obsolete and irrelevant, the Liberal Party, with or without the Duttons of this world, must soon begin to transform to 21st century life or face extinction (“Dutton must pass Harbour test”, February 3). The fresh Labor government has shaped up well so far but needs a viable and intelligent opposition for our democracy to function as we all would wish. Why is no one rising to the challenge? Brian Haisman, Winmalee\n Nick Bryant is one among many who point out the difficulties ahead for Peter Dutton as leader of the Liberal party. I wonder if a 2023 version of Don Chipp will emerge from the moderate malcontents of the party. John Bailey, Canterbury\n Does Peter Dutton know what’s really “an attack on our society”? He plays the divide and conquer card while aiming to appeal to the tone-deaf. Trying to ignite a fire when none need to exist. It’s well past time to pay our due to our First Nations on whose land we live. It’s time to grow up. Francesca Stahlut, Armidale\n \n\nOnly stand up for a standing ovation\n Theatregoers standing for the curtain call ovation are a minor irritation (Letters, February 3). The real problem punters are the concertgoers who stand in their seats for half of the show. John Swanton, Coogee\n Over the decades I have attended countless music concerts and many had the audience give a standing ovation. There was one concert where the crowd got to its feet at the end not in appreciation but to demand the band return to the stage as their performance had been so short.\n When it appeared they had no intention of playing more, the audience took to throwing cushions and ripping seats out to throw at the stage in disgust. This was Elvis Costello’s first concert in Sydney in 1978 at the Regent theatre; his future ones were much longer. Con Vaitsas, Ashbury\n \n \n\nLooking for a three-headed emu in a haystack\n Good news that the radioactive pod was located in Western Australia, but I await the first appearance of a three-headed emu (Letters, February 3). Mark Berg, Caringbah South\n \n\nPut ‘woke’ back to sleep\n Could we please put “woke” back to sleep. It has become another boring, pointless word that spoils a good argument or description. Kath Maher, Lidcombe\n \n\nPostscript\n Kim Crawford of Springwood was one of a number of letter writers who claimed to have double-checked the date on Friday’s Herald front page. So confounding were the headlines - Cardinal George Pell described as a “martyr” and a “saint of our times”; the former NSW deputy premier eyeing off the newly vacated job of ClubsNSW chairman, and living up to his nickname of “pork Barilaro” - that correspondents thought they’d woken up on April 1.\n George Fishman of Vaucluse was one of only a few who wrote that “whatever side you are on, the funeral oration by Tony Abbott hailing George Pell was one for the ages”. Not surprisingly, the majority of writers were scathing of the former PM’s statement that “Pell was the greatest man I’ve ever known”, many agreeing with Peter Kamenyitzky of Castle Hill that Abbott “clearly lives in a parallel universe to most of us”, while others suggested he needs to meet more men.\n Others, like Sue Burton of Mosman, were critical of Abbott turning “a wilful blind eye to Pell’s own wilful and well documented blind eye to the abuse of children”.\n “I respectfully suggest that John Barilaro now be awarded the title of ‘the human headline’ (with apologies to Derryn Hinch),“wrote Lorraine Hickey of Green Point. “So many juicy stories, so little time.” Phil Bradshaw of Naremburn was so bewildered at the former minister’s ambition, he wondered whether “Pork Barilaro” would consider putting his hand up for POTUS in 2024. Pat Stringa, letters editor\n To submit a letter to The Sydney Morning Herald, email letters@smh.com.au. Click here for tips on how to submit letters. The Opinion newsletter is a weekly wrap of views that will challenge, champion and inform. Sign up here.\n'

texts = text_splitter.create_documents([page_content])
for t in texts:
    print(t)
    print()
    print()

page_content='HeadLine: Gobsmacked by Barilaro’s behaviour? Join the club
Content: Just what ClubsNSW needs: as it is trying to convince NSW voters that it has a good heart, a bid by John Barilaro to take over the helm (“Barilaro bids to become clubs boss”, February 3). Here is a man who achieved infamy by repeatedly demonstrating questionable judgment throughout his political life. Barilaro’s attempt to take over from Josh Landis should be consigned to the same rubbish bin that holds his bid for appointment as NSW trade commissioner to New York. Kim Woo, Mascot'


page_content='I read the article about John Barilaro’s utterly unconscionable behaviour, which effectively deprived most of the Labor electorates of their emergency bushfire funding, with smoke coming out of my ears (“Barilaro’s office in fire funding storm”, February 3). The admitted pork barrelling by this government was a vote-buying strategy. Such reprehensible action needs way more than the usual slap on the wrist, so i

In [49]:
if 1==1:
    #Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
       #separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)   

NameError: name 'documents' is not defined

In [24]:
import tiktoken

def truncate_text_tokens(text, encoding_name=EMBEDDING_ENCODING, max_tokens=EMBEDDING_CTX_LENGTH):
    """Truncate a string to have `max_tokens` according to the given encoding."""
    encoding = tiktoken.get_encoding(encoding_name)
    return encoding.encode(text)[:max_tokens]

ModuleNotFoundError: No module named 'tiktoken'

In [25]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.4 MB/s eta 0:00:0000:010:01


In [26]:
from itertools import islice

def batched(iterable, n):
    """Batch data into tuples of length n. The last batch may be shorter."""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

In [32]:

import tiktoken
def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunks_iterator = batched(tokens, chunk_length)
    yield from chunks_iterator

In [40]:
import numpy as np

EMBEDDING_MODEL = 'textembedding-gecko'
EMBEDDING_CTX_LENGTH = 2048
EMBEDDING_ENCODING = 'cl100k_base'


def len_safe_get_embedding(text, model=EMBEDDING_MODEL, max_tokens=EMBEDDING_CTX_LENGTH, encoding_name=EMBEDDING_ENCODING, average=True):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, encoding_name=encoding_name, chunk_length=max_tokens):
        #chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))
       
    
    # if average:
    #     chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
    #     chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)  # normalizes length to 1
    #     chunk_embeddings = chunk_embeddings.tolist()
    return chunk_lens #chunk_embeddings

In [30]:
text='HeadLine: Gobsmacked by Barilaro’s behaviour? Join the club\nContent: Just what ClubsNSW needs: as it is trying to convince NSW voters that it has a good heart, a bid by John Barilaro to take over the helm (“Barilaro bids to become clubs boss”, February 3). Here is a man who achieved infamy by repeatedly demonstrating questionable judgment throughout his political life. Barilaro’s attempt to take over from Josh Landis should be consigned to the same rubbish bin that holds his bid for appointment as NSW trade commissioner to New York. Kim Woo, Mascot\n I read the article about John Barilaro’s utterly unconscionable behaviour, which effectively deprived most of the Labor electorates of their emergency bushfire funding, with smoke coming out of my ears (“Barilaro’s office in fire funding storm”, February 3). The admitted pork barrelling by this government was a vote-buying strategy. Such reprehensible action needs way more than the usual slap on the wrist, so it’ll be interesting to see what happens next. Anne Ring, Coogee\n To exacerbate people’s misery, and politicise it, is the lowest of the low. The relief recovery program is there to ensure all affected by the Black Summer fires get assistance. Barilaro shows us – again and again – how little compassion he has for the people of NSW. Daniela Catalano, Haberfield\n Sadly, the revelations show two things: first, that the egregious former minister for John Barilaro appears ready to continue to serve the people of NSW in any capacity – provided it has a six-figure salary attached. Second, that while in office he didn’t have the nous to understand how pork-barrelling works. The idea is to get people to vote for you, so it is logically directed at seats you don’t have, rather than ones you do. Unless, of course, he realised the Coalition was (is) so on the nose that it needed to shore-up those seats it already held. Or maybe both of these cases are true. Greg Oehm, Robertson\n Why is there no punishment for pork barrelling? In public life, fraudulent behaviour is punishable under law. For politicians, it’s business as usual. Forget about improved sporting facilities, safer car parks and appropriate disaster relief. So long as these actions come without consequence, nothing will change. Sam Kent, Hunters Hill\n I pondered long and hard in an attempt to predict what Barilaro might include in his job application for ClubsNSW. All I could dredge up were greyhounds, koalas, brumbies, threats to blow up the government, overseas trade appointments and, then, Thursday’s revelation of his interference in the bushfire recovery program. When will our failed politicians realise that they are unemployable? Even by ClubsNSW. Rhonda Seymour, Castle Hill\n Quite appropriate really – a man who directed desperately needed bushfire recovery funds away from ALP seats now wants to run an organisation that exists on the back of problem gamblers and illegal money laundering. Sounds like the ethics will be a perfect fit. Michael McMullan, Avoca Beach\n Is it any wonder that Barilaro denied bushfire recovery funding to the Blue Mountains? Apart from it being a Labor seat, he once referred to koalas as tree rats. The man has no love of the bush. Sally Spurr, Lane Cove\n The former deputy premier is reportedly proud of his nickname Pork Barilaro. His arrogance is breathtaking, and the revelation that the joint federal and state emergency bushfire funds were distributed on partisan grounds under his direction is contemptuous. Why am I not surprised that he has now put his hand up for the recently vacated ClubsNSW job? His intervention in the emergency bushfire recovery program needs to be immediately reviewed by ICAC. Peter Neufeld, Mosman\n I suggest that regardless of which party wins in March it considers appointing a Chief Pub Tester for major policy and decision-making. The beauty of the role is that it could be filled by 99 per cent of the population, and the only equipment the chief would require is an egg timer, a bar stool and a beer or G&T. Monique Darcy, Davidson\n The integrity and transparency of the Perrottet government can be summed up in one word: Barilaro. Michael Petras, Thornleigh\n It’s hard to decide whether I feel utter contempt for the cynical interference in disaster relief or just bewilderment at such delusional ambition? Has Barilaro considered putting his hand up for POTUS in 2024? Phil Bradshaw, Naremburn\n Good old “Pork Barilaro”, the gift that keeps giving to NSW Labor. Rob Phillips, North Epping\n I’m buying up popcorn. The Barilaro ICAC will be a corker. Dorothy Kamaker, Whale Beach\n Divisive Pell not a martyr, and no saint Tony Abbott called Cardinal George Pell “a saint of our times” and a victim of a modern-day crucifixion (“As Pell the man is laid to rest, Pell the martyr rises”, February 3). As I understand from my 13 years of Catholic education, Jesus was crucified for threatening and challenging the entrenched power structure and calcified ideologies and attitudes of the church hierarchy, not for propping it up. Monica Oppen, Stanmore \n Abbott’s grasp of the requirements for sainthood and of the array of great Catholics that Australia has produced helps me to understand even further his time as prime minister. Sister Susan Connelly, Lakemba \n My condolences to anyone praying for any ongoing meaningful contribution from Abbott to Australia. Ben Dryza, Newtown \n Abbott said “George Pell was the greatest man I’ve ever known”. Yet no knighthood. Mark Kilminster, Castle Hill \n “Warrior”, “soldier of truth”, “saint”, “hero”. Gandhi? Nelson Mandela? Martin Luther King? No. George Pell. Seriously? Julia Booth, Westleigh \n By far the most significant comment at the funeral was Abbott’s; that Pell should never have been investigated without a complaint; he should never have been charged without corroborating evidence, and should never have been convicted without a plausible case, as the High Court eventually realised. Laurie Le Claire, Epping \n Such delusions of grandeur. The deliberately divisive Pell was no more a saint than Abbott was a leader. Anne Garvan, Chatswood West\n The last time I checked, Saint Mary Mackillop is clearly more highly revered than a late cardinal. Saint Mary Mackillop is the greatest Catholic Australia has ever produced. Zara Tai, Minchinbury\n The question is, has Pell travelled northwards or southwards? David Boyd, Bondi Beach\n Pell incites strong opinions among his supporters and haters, even in death. Let us all move on with his passing, but may the Catholic Church move with integrity, compassion and justice. If the Church always tries to put the institution above the people, the institution, even after 2000 years, will not survive. Vincent Wong, Killara\n Dear Tony Abbott,How dare you turn a wilful blind eye to Cardinal Pell’s own wilful and well documented blind eye to the abuse of children. Sue Burton, Mosman\n The man Abbott describes as the “greatest I’ve ever known” was someone found by a royal commission to have protected paedophiles. What does this say about Abbott?\n Still, it’s no surprise to see a conga line of Liberal luminaries now waxing so lyrical about Cardinal Pell. In Pell they found a fellow right wing political operative par excellence, who brought the power and prestige of the Church to the project of reactionary conservatism. Michael Hinchey, New Lambton \n Clearly, Abbott lives in a parallel universe to most of us. Peter Kamenyitzky, Castle Hill \n Interesting to see the fawning adoration by Tony Abbott at the memorial service for George Pell. Including of course that he was a victim of a media witch hunt against him and the Catholic Church in general. George Pell presided over an administration that did little to redress the victims of sexual abuse nor did it acknowledge what future safeguards have been put in place. His administration was a failure. His only interest was protecting the Church and its money. His line at the royal commission, “It was of little interest to me”, perhaps shows the true colours of the man. Stephen Trevarrow, New Farm (Qld)\n How do we reconcile Tony Abbott’s proclamation that Cardinal George Pell is a “saint for our times” with Pell’s own statement that multiple claims of child sexual abuse perpetrated by Catholic priests under his watch were of not much interest to him? The man is hardly a martyr if the definition is “a person who voluntarily suffers death as the penalty of witnessing to and refusing to renounce a religion”. Meg Pickup, Ballina \n Not one letter writer has shown even a drop of human kindness (Letters, February 3). The man, a beacon of endeavour and achievement, was an early crusader in the fight against sexual abuse in the church, subsequently deemed inadequate, but revolutionary in its time. Pell endured horrifically biased treatment from the ABC and the Victorian police, tolerated a dud rap as a criminal, and was eventually totally vindicated, yet, rather than bitterness or hatred, he displayed only his enduring faith in his God.\n How was Pell not a giant of a man? Flawed yes, but none of us is perfect, and he didn’t ever deserve the viciousness he faced. Convicted serial killers are shown more tolerance. Rosemary O’Brien, Ashfield\n Bernini explained that since St. Peter’s is “the mother church of nearly all the others, it had to have colonnades, which would show it as if stretching out its arms maternally to receive Catholics, so as to confirm them in their faith, heretics to reunite them to the church, and infidels, to enlighten them in the true faith.” Pell’s funeral made plain for all the church in Australia has closed its arms to the wounded, those it harmed, ignored their cries of pain, and in so doing, failed to follow Jesus’ admonition to welcome and comfort the injured and hold them close, invite them in to St Mary’s Cathedral and acknowledge its crimes against the young innocents. Instead, it denied the findings of a royal commission, denied their very existence. The beginning of the road to redemption is to first admit your sins. This week, the Catholic Church did the opposite, it said nothing happened, there were no victims and proved it is too arrogant and out of touch, it can only deny its acts by washing away the blood of the young innocent. Philip Drew, Annandale\n Thank goodness for former PM John Howard, Tony Abbott, Peter Dutton, Matt Canavan, and Dan Tehan for standing up to witness the passing of Cardinal Pell.It is disappointing our other current political leaders were too small-minded, self-serving, and “go with the flow” to honour Pell who was used as a scapegoat, and suffered 404 days in gaol for a crime he did not commit. But he is in a better place now, thank God. Elizabeth Vickers, Maroubra\n  Nevermind the monarchists, get the royals off our notes Does the decision to replace the late Queen’s head with an Indigenous design on a single banknote really constitute a poke in the eye to monarchists, or is it simply an attempt to acknowledge the nation’s First People on our currency (“Timing of new $5 note without King could be better”, February 3)? It seems an over reaction to a minor change that will probably go unnoticed by most of the population. After all, how many of us could accurately describe exactly what is on each banknote without looking? Merona Martin, Meroo Meadow\n I don’t understand why the dysfunctional Windsor family are on Australia’s currency anyway. James Duggan, Hunters Hill\n The monarchists seem so sad over the $5 note business. Why not cheer them up by having more royals on our money, reminding us colonials how great the monarchy has been. Put Charles on the fiver and Andrew on the $10. The $20 could have the former Edward VIII and Wallace. The $50 would be perfect for Henry VIII, circled by his five wives. Nothing less than the $100 note would do for Prince Harry. Richard Macey, Pendle Hill \n I applaud the decision not to issue a $5 note with King Charles on it. They are obviously waiting until after the coronation; any balding man looks better wearing a hat or a large crown. Noel Mills, Avalon Beach\n Better pay the best cure for GP crisis The government’s approach is novel: leave the cause untreated and see if the patient survives (“Doctor pay unaddressed in report”, February 3). When the main cause of the crisis in general practice is that the bulk-billing Medicare rebate is about half of where it should have been had it kept pace with inflation over the last 30 years – making bulk-billing economically unviable and leading medical graduates to pursue other paths where their expertise is more valued – the government seems to want to waste money on doing everything it can to avoid actually treating the cause. The prognosis isn’t good. Alan Garrity, North Narrabeen \n A close family friend has had his very successful inner-city GP practice up for sale for three years with no buyers and has now been forced to simply shut it down. Payments to GPs have not increased markedly for many years unlike their business overheads, resulting in marginal profitability for doctors often working in excess of 60 hours a week. What a sad commentary on our modern society when highly trained and respected GPs, whose opinions often determine the length and quality of our lives, struggle to make a living while tradies earn $150 an hour and drive around in $100,000 utes. Tony Snellgrove, Tumbulgum\n  Inequality needs new thinking Decades of neo-liberal economics along with the slow burn of fiscal austerity have driven the relentless selling-off of government instrumentalities and the ransacking of many vital government departments (“Shrill critics of Chalmers’ essay are missing the point,” February 3). It provided previous governments with the revenue to win votes through tax cuts and nepotistic funding rorts. It made winners of the well-heeled, while more of us became steadily less and less well off.\n Inequality has now reached epidemic proportions and areas such as education, health, housing, aged care and transport are in fiscal crisis and desperate for funding. For decades, we have been sold the lie that taxes are a burden on our economy and to be feared. It is time we considered a more equitable and inclusive way of improving the wealth of our nation. Bruce Spence, Balmain\n We now have a treasurer who writes an essay offering new ideas to create a fairer, better society for all Australians. His ideas are progressive, thought-provoking and challenging. The voices that criticise him, immediately label him a communist and a real danger to corporate Australia. The attitude by those that have, against those that have not, are predictable, archaic, self-centred and selfish. Jim Chalmers is thinking about our future not the next election, surely that’s worth consideration. Geoff Hermon, Maraylya\n Baffling gas decision One of the few good decisions made by Scott Morrison is likely to be overturned (“Scott Morrison’s decision to stop controversial NSW gas field to be overturned”, smh.com.au, February 3). Whatever Morrison’s motives, this was a decision which went some way to address climate change concerns. It is now likely to be overturned with the help of a federal government that claims to care about climate change and global warming. The reason for this decision by the federal government is, apparently, the potential cost of compensation to the gas companies. Once again money trumps public interest.As a long-term Labor supporter I am baffled and deeply troubled by their approach to this issue. Edward Quinn, St Ives \n The Wounded Kangaroo Alan Joyce’s perpetual apologies for Qantas failings wear thin when your QF27 luggage does not arrive in Buenos Aires three days before departing an Antarctic voyage (“Despite the hype, Qantas turnbacks are a sign of strong safety”, February 3). No apologies. No explanation. And no expedition gear. Just anxiety.\n Unfortunately, Qantas no longer brings an offering to market which is as certain as it may be safe.\n It’s time for a management rethink because, right now, the Spirit of Australia days are over. Any airline will do. Stephen Goddard, Bowral\n Classroom conundrum Decisions are complex, not binary. Should single sex schools exist? It depends (“Enrolments at public boys’ high schools in steep decline”, February 3).\n For some children they work. For others not. It is about the makeup of the individual, not social engineering. My son went to Homebush Boys High and it worked for him. Would he have done better in a co-ed school? We will never know.\n The pendulum is swinging towards co-ed schools. I predict in a couple of decades it will start swinging the other way. Why do we think there has to be one answer? Neville Turbit, Russell Lea\n Dutton must grow up Ossified, obsolete and irrelevant, the Liberal Party, with or without the Duttons of this world, must soon begin to transform to 21st century life or face extinction (“Dutton must pass Harbour test”, February 3). The fresh Labor government has shaped up well so far but needs a viable and intelligent opposition for our democracy to function as we all would wish. Why is no one rising to the challenge? Brian Haisman, Winmalee\n Nick Bryant is one among many who point out the difficulties ahead for Peter Dutton as leader of the Liberal party. I wonder if a 2023 version of Don Chipp will emerge from the moderate malcontents of the party. John Bailey, Canterbury\n Does Peter Dutton know what’s really “an attack on our society”? He plays the divide and conquer card while aiming to appeal to the tone-deaf. Trying to ignite a fire when none need to exist. It’s well past time to pay our due to our First Nations on whose land we live. It’s time to grow up. Francesca Stahlut, Armidale\n Only stand up for a standing ovation Theatregoers standing for the curtain call ovation are a minor irritation (Letters, February 3). The real problem punters are the concertgoers who stand in their seats for half of the show. John Swanton, Coogee\n Over the decades I have attended countless music concerts and many had the audience give a standing ovation. There was one concert where the crowd got to its feet at the end not in appreciation but to demand the band return to the stage as their performance had been so short.\n When it appeared they had no intention of playing more, the audience took to throwing cushions and ripping seats out to throw at the stage in disgust. This was Elvis Costello’s first concert in Sydney in 1978 at the Regent theatre; his future ones were much longer. Con Vaitsas, Ashbury\n  Looking for a three-headed emu in a haystack Good news that the radioactive pod was located in Western Australia, but I await the first appearance of a three-headed emu (Letters, February 3). Mark Berg, Caringbah South\n Put ‘woke’ back to sleep Could we please put “woke” back to sleep. It has become another boring, pointless word that spoils a good argument or description. Kath Maher, Lidcombe\n Postscript Kim Crawford of Springwood was one of a number of letter writers who claimed to have double-checked the date on Friday’s Herald front page. So confounding were the headlines - Cardinal George Pell described as a “martyr” and a “saint of our times”; the former NSW deputy premier eyeing off the newly vacated job of ClubsNSW chairman, and living up to his nickname of “pork Barilaro” - that correspondents thought they’d woken up on April 1.\n George Fishman of Vaucluse was one of only a few who wrote that “whatever side you are on, the funeral oration by Tony Abbott hailing George Pell was one for the ages”. Not surprisingly, the majority of writers were scathing of the former PM’s statement that “Pell was the greatest man I’ve ever known”, many agreeing with Peter Kamenyitzky of Castle Hill that Abbott “clearly lives in a parallel universe to most of us”, while others suggested he needs to meet more men.\n Others, like Sue Burton of Mosman, were critical of Abbott turning “a wilful blind eye to Pell’s own wilful and well documented blind eye to the abuse of children”.\n “I respectfully suggest that John Barilaro now be awarded the title of ‘the human headline’ (with apologies to Derryn Hinch),“wrote Lorraine Hickey of Green Point. “So many juicy stories, so little time.” Phil Bradshaw of Naremburn was so bewildered at the former minister’s ambition, he wondered whether “Pork Barilaro” would consider putting his hand up for POTUS in 2024. Pat Stringa, letters editor\n To submit a letter to The Sydney Morning Herald, email letters@smh.com.au. Click here for tips on how to submit letters. The Opinion newsletter is a weekly wrap of views that will challenge, champion and inform. Sign up here.\n'

In [41]:
x=len_safe_get_embedding(text)

In [37]:
x

[2048, 2048, 387]

In [246]:
from vertexai.preview.batch_prediction import BatchPredictionJob
for job in BatchPredictionJob.list():
    print(job.to_dict())

In [289]:
from google.cloud import bigquery
client = bigquery.Client()

# List jobs for the project
jobs = client.list_jobs()

for job in jobs:
    # try:
    #     print(job.statement_type)
    # except:
    #     continue
    if (job.job_type == 'query' and job.statement_type == 'ML.GENERATE_EMBEDDING') or (job.job_type == 'query' and 'ML.GENERATE_EMBEDDING' in job.query):
        print(f"Job ID: {job.job_id}, State: {job.state}, Created: {job.created},  Ended: {job.ended}   )
              break
 

SyntaxError: unterminated string literal (detected at line 13) (2251090377.py, line 13)

In [85]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_community import BigQueryLoader
from datetime import datetime
import logging
from google.cloud import bigquery
import os, json
   
def create_dataset(project_id,dataset_id,region_id):
    """
        Create a dataset
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str region_id:  name of the region under whcih the dataset should be created.               
    """
    
    client = bigquery.Client(project_id)
    print(project_id)
    # Check if the dataset exists
    try:
        dataset = client.get_dataset(f"{project_id}.{dataset_id}")  # Make an API request.
        print(f"Dataset '{dataset_id}' already exists.")
    except :
        # If the dataset does not exist, create it
        
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
        dataset.location = region_id # Set your desired location

        # Create the dataset
        dataset = client.create_dataset(dataset)  # Make an API request.
        print(f"Dataset '{dataset_id}' created successfully.")
        
 
def create_table(project_id,dataset_id,table_id):
    
    """
        Create a table 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table.
        Returns:
            list[bigquery.SchemaField] schema: table schema- list of columns
             
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("request_id", "STRING", mode="REQUIRED"),
        #bigquery.SchemaField("original_content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("combined_id", "STRING", mode="REQUIRED"),
       # bigquery.SchemaField("media_type", "STRING", mode="NULLABLE"),
        #bigquery.SchemaField("path", "STRING", mode="NULLABLE"),
       # bigquery.SchemaField("test_metadata", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("chunk", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("process_time", "DATETIME", mode="REQUIRED"),
    ]
                

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
        print(f"Table '{full_table_id}' already exists.")
        
        # Drop the table if exist
        query = f"DROP TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' dropped successfully.")
            
            # recreate the table
            table = bigquery.Table(full_table_id, schema=schema)

            # Create the table
            table = client.create_table(table)  # Make an API request.
            print(f"Table '{full_table_id}' created successfully.")
        
        except Exception as e:
            print(f"Error dropping/ recreating table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema


def load_and_split_docs (source_query_str: str,project_id: str, metadata_columns: list[str], page_content_columns: list[str], chunk_size: int,chunk_overlap: int ):
    
    """
        Load data from biquery table and chunk them 
        
        Args:
           str source_query_str: source query string
           str project_id: project id
           list[str] metadata_columns: list of columns from the source table 
           list[str] page_content_columns:  list of page content columns from the source table
           int chunk_size: chunk size in character
           int chunk_overlap: chunk overlap in character
           
        Returns:
            list[documents] doc_splits: list of splitted documents
             
    """
    
      # Load the data
    loader = BigQueryLoader(
        query=source_query_str, project=project_id, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )
    
    documents = []
    documents.extend(loader.load())
    
    print(f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       #separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)   
    
    return doc_splits

def load_into_bq(table_prefix: str,project_id: str,dataset_id: str, region: str, data: list,  version: int):  
    """
        Create a job to load data into big query table and wait for the job to finish 
        
        Args:
           str table_prefix: the prefix of biquery table name
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           list(json) data: list of json records that should be inserted into table.
     
             
    """    
    client = bigquery.Client(project_id)    
   
    
    #create table new if does not exist
    table=f"{table_prefix}_{version}"
    table_schema=create_table(project_id,dataset_id,table)
    #push the data into the table
    table_id = f"{project_id}.{dataset_id}.{table}"
    dataset  = client.dataset(dataset_id)
    table = dataset.table(table)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    job_config.schema = table_schema
    job = client.load_table_from_json(data, table, job_config = job_config)
              
    #wait for the job to finish
    job.result()
    # Get job status   
                
    if job.state == 'DONE':
        if job.error_result:
             print(f"Job {job.job_id} for {table_id} failed with error: {job.error_result}")
             raise Exception("Sorry, no numbers below zero")
        else:
             print(f"Job {job.job_id} completed successfully. For "+ table_id)
    else:
            print(f"Job {job.job_id} for {table_id} is still in progress.")
                


def chunk_bq_content(request_args):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and load the result into bigquery
             
    """
    status=''   


    project_id=  request_args['project_id']
    dataset_id=  request_args['dataset']
    table= request_args['table']
    region= request_args['region']
    metadata_columns= [col.strip() for col in  str(request_args['metadata_columns']).split(',') ]
    article_page_content_columns= [col.strip() for col in str(request_args['article_page_content_columns']).split(',') ]
    video_page_content_columns= [col.strip() for col in str(request_args['video_page_content_columns']).split(',') ]
    image_page_content_columns= [col.strip() for col in str(request_args['image_page_content_columns']).split(',') ]
    article_source_query_str= request_args['article_source_query_str']
    video_source_query_str= request_args['video_source_query_str']
    image_source_query_str= request_args['image_source_query_str']            
    #separators= "\n" if str(request_args['separators'])=="" else str(request_args['separators']).split(',') 
    chunk_size= 1000 if str(request_args['chunk_size']) in ["None",""] else int(str(request_args['chunk_size']))  
    chunk_overlap=100 if str(request_args['chunk_overlap']) in ["None",""] else int(str(request_args['chunk_overlap'])) 
    max_prompt_count_limit=20000 if str(request_args['max_prompt_count_limit']) in ["None",""] else int(str(request_args['max_prompt_count_limit'])) 
  

    #load and split articles       
    doc_splits= load_and_split_docs(article_source_query_str,project_id, metadata_columns, article_page_content_columns, chunk_size,chunk_overlap )       
    print(f"Article contents loaded and splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}\n")
 
    #load and split videos    
    doc_splits= doc_splits+ load_and_split_docs(video_source_query_str,project_id, metadata_columns, video_page_content_columns, chunk_size,chunk_overlap )        
    print(f"Video contents loaded and splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}\n")
   
    #load and split images 
    doc_splits= doc_splits+ load_and_split_docs(image_source_query_str,project_id, metadata_columns, image_page_content_columns, chunk_size,chunk_overlap )        
    print(f"Image contents loaded and splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}\n")

    
    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata[metadata_columns[0]]
    rows_to_insert=[]
    #request_date=datetime.today().strftime('%Y_%m_%d') 
    now = datetime.now()    
    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)
   
    max_index=max_prompt_count_limit #maximum number of requests in a batch
    record_count=0
    prefix=f"{table}" 
    job_list=[]
    job_execution_result={}
    #laod data into bigquery table
    for idx, split in enumerate(doc_splits):
            split.metadata["process_time"]=now
            if prev==split.metadata[metadata_columns[0]]:
               split.metadata["chunk"] = chunk_idx      
            else:
                chunk_idx=0
                split.metadata["chunk"] = chunk_idx
                prev=split.metadata[metadata_columns[0]]
                
            chunk_idx +=1
            version=idx // max_index
            request_id = prefix+'_'+str(version)
            
            if chunk_idx==1:                
                 content=split.page_content  
            else:                  
                  content="Content"+": "+split.page_content 
                
            rows_to_insert.append(
                               {  "request_id":  request_id  , 
                                   "combined_id": split.metadata[metadata_columns[0]], 
                                   "process_time":split.metadata["process_time"].isoformat(),
                                   "content": content,#.replace(page_content_columns[0]+":", "", 1).strip(),
                                   #"original_content": split.metadata["content"],
                                   "chunk": split.metadata["chunk"],
                                   #"media_type": split.metadata["media_type"],
                                   #"path": split.metadata["path"],
                                   #"test_metadata": split.metadata["test_metadata"]                            

                                  }
                                         )
            
            if (idx+1) % max_index==0:  
                load_into_bq(prefix,project_id,dataset_id, region, rows_to_insert,version)
                #moving to next batch
                record_count=record_count+len(rows_to_insert)
                rows_to_insert=[]   
               
                
    if  len(rows_to_insert)>0:       
        load_into_bq(prefix,project_id,dataset_id, region, rows_to_insert,version)
        #moving to next batch
        record_count=record_count+len(rows_to_insert)
        rows_to_insert=[]   
                
    
  
    return   {'status':'SUCCESS', 'record_count':record_count,'count_of_tables':version+1 }



if __name__ == "__main__":   

        request_args={}
        if  'project_id' in os.environ:
            request_args['project_id']= os.environ.get('project_id')
            
        if  'dataset' in os.environ:
            request_args['dataset']= os.environ.get('dataset')
            
        if  'table' in os.environ:
            request_args['table']= os.environ.get('table')
            
        if  'region' in os.environ:
            request_args['region']= os.environ.get('region')
            
        if  'metadata_columns' in os.environ:
            request_args['metadata_columns']= os.environ.get('metadata_columns')
            
        if  'article_page_content_columns' in os.environ:
            request_args['article_page_content_columns']= os.environ.get('article_page_content_columns')
            
        if  'video_page_content_columns' in os.environ:
            request_args['video_page_content_columns']= os.environ.get('video_page_content_columns')
        
        if  'image_page_content_columns' in os.environ:
            request_args['image_page_content_columns']= os.environ.get('image_page_content_columns')
            
        if  'article_source_query_str' in os.environ:
            request_args['article_source_query_str']= os.environ.get('article_source_query_str')
            
        if  'video_source_query_str' in os.environ:
            request_args['video_source_query_str']= os.environ.get('video_source_query_str')
           
        if  'image_source_query_str' in os.environ:
            request_args['image_source_query_str']= os.environ.get('image_source_query_str')
            
        if  'chunk_size' in os.environ:
            request_args['chunk_size']= os.environ.get('chunk_size')
            
        if  'chunk_overlap' in os.environ:
            request_args['chunk_overlap']= os.environ.get('chunk_overlap')
            
        if  'max_prompt_count_limit' in os.environ:
            request_args['max_prompt_count_limit']= os.environ.get('max_prompt_count_limit')

    
        chunk_bq_content(request_args)
            
          
 

KeyError: 'project_id'

In [86]:
print("Describe this video from period --- seconds to --- seconds.\n Focus on key themes, what people are talking about, names of people who appeared and who are discussed and key locations. Highlight any brands, company names or logos that you see. Give me at least 4000 words describing it.  Do not add any extra text to the description. Organize the description in the following format: \n**Category**\n \n**DetailedDescriptionOfEventsAndConversations**\n \n**BrandsCompanyNamesLogos**\n \n**KeyLocationsAndScenes**\n \n**KeyThemes**\n \n**PeopleAppearingAndMentioned**\n When describing the DetailedDescriptionOfEventsAndConversations, consider the following instructions for specific video types: * **News:** Pay close attention to transitions, graphics, and on-screen text. * **TV Shows:** Describe facial expressions, body language, appearances, and overall mood. * **Live Sports Events:** Focus on key moments, like goals or fouls, and describe the overall flow and momentum of the game. * **News Analyses:** Identify different perspectives, arguments, and supporting evidence.")

Describe this video from period --- seconds to --- seconds.
 Focus on key themes, what people are talking about, names of people who appeared and who are discussed and key locations. Highlight any brands, company names or logos that you see. Give me at least 4000 words describing it.  Do not add any extra text to the description. Organize the description in the following format: 
**Category**
 
**DetailedDescriptionOfEventsAndConversations**
 
**BrandsCompanyNamesLogos**
 
**KeyLocationsAndScenes**
 
**KeyThemes**
 
**PeopleAppearingAndMentioned**
 When describing the DetailedDescriptionOfEventsAndConversations, consider the following instructions for specific video types: * **News:** Pay close attention to transitions, graphics, and on-screen text. * **TV Shows:** Describe facial expressions, body language, appearances, and overall mood. * **Live Sports Events:** Focus on key moments, like goals or fouls, and describe the overall flow and momentum of the game. * **News Analyses:** Ide

In [88]:
from typing import List

from google.cloud import iam_admin_v1
from google.cloud.iam_admin_v1 import types


def list_service_accounts(project_id: str) -> List[iam_admin_v1.ServiceAccount]:
    """
    Get list of project service accounts.

    project_id: ID or number of the Google Cloud project you want to use.
    """

    iam_admin_client = iam_admin_v1.IAMClient()
    request = types.ListServiceAccountsRequest()
    request.name = f"projects/{project_id}"

    accounts = iam_admin_client.list_service_accounts(request=request)
    return accounts.accounts

In [89]:
list_service_accounts('nine-quality-test')

[name: "projects/nine-quality-test/serviceAccounts/494586852359-compute@developer.gserviceaccount.com"
project_id: "nine-quality-test"
unique_id: "117381591874063009905"
email: "494586852359-compute@developer.gserviceaccount.com"
display_name: "Compute Engine default service account"
etag: "01021920"
oauth2_client_id: "117381591874063009905"
, name: "projects/nine-quality-test/serviceAccounts/nine-quality-test@appspot.gserviceaccount.com"
project_id: "nine-quality-test"
unique_id: "113671129977640505121"
email: "nine-quality-test@appspot.gserviceaccount.com"
display_name: "App Engine default service account"
etag: "01021920"
oauth2_client_id: "113671129977640505121"
, name: "projects/nine-quality-test/serviceAccounts/workflowrunner@nine-quality-test.iam.gserviceaccount.com"
project_id: "nine-quality-test"
unique_id: "102292657621919090032"
email: "workflowrunner@nine-quality-test.iam.gserviceaccount.com"
display_name: "WorkflowRunner"
etag: "01021920"
description: "service account to r

In [96]:
!gcloud projects get-iam-policy $(gcloud config get-value project) --flatten=bindings[].members --format=table(bindings.role,bindings.members) --filter=bindings.members:service-*@gcp-sa-aiplatform.iam.gserviceaccount.com

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `gcloud projects get-iam-policy $(gcloud config get-value project) --flatten=bindings[].members --format=table(bindings.role,bindings.members) --filter=bindings.members:service-*@gcp-sa-aiplatform.iam.gserviceaccount.com'


In [95]:
!(gcloud config get-value project) --flatten=bindings[].members --format=table(bindings.role,bindings.members) --filter=bindings.members:service-*@gcp-sa-aiplatform.iam.gserviceaccount.com

/bin/bash: -c: line 1: syntax error near unexpected token `--flatten=bindings[].members'
/bin/bash: -c: line 1: `(gcloud config get-value project) --flatten=bindings[].members --format=table(bindings.role,bindings.members) --filter=bindings.members:service-*@gcp-sa-aiplatform.iam.gserviceaccount.com'


In [94]:
! gcloud services list --enabled | grep aiplatform

aiplatform.googleapis.com            Vertex AI API


In [97]:
!gcloud iam service-accounts list

DISPLAY NAME                            EMAIL                                                       DISABLED
Compute Engine default service account  494586852359-compute@developer.gserviceaccount.com          False
App Engine default service account      nine-quality-test@appspot.gserviceaccount.com               False
WorkflowRunner                          workflowrunner@nine-quality-test.iam.gserviceaccount.com    False
functionrunner                          functionrunner@nine-quality-test.iam.gserviceaccount.com    False
Cloud Run to access Vertex AI APIs      vertex-ai-caller@nine-quality-test.iam.gserviceaccount.com  False


In [98]:
! gcloud iam service-accounts list

DISPLAY NAME                            EMAIL                                                       DISABLED
Compute Engine default service account  494586852359-compute@developer.gserviceaccount.com          False
App Engine default service account      nine-quality-test@appspot.gserviceaccount.com               False
WorkflowRunner                          workflowrunner@nine-quality-test.iam.gserviceaccount.com    False
functionrunner                          functionrunner@nine-quality-test.iam.gserviceaccount.com    False
Cloud Run to access Vertex AI APIs      vertex-ai-caller@nine-quality-test.iam.gserviceaccount.com  False


In [103]:
! gcloud iam service-accounts get-iam-policy service-301679871630@gcp-sa-aiplatform.iam.gserviceaccount.com --format=json

ERROR: (gcloud.iam.service-accounts.get-iam-policy) PERMISSION_DENIED: Permission 'iam.serviceAccounts.getIamPolicy' denied on resource (or it may not exist). This command is authenticated as 494586852359-compute@developer.gserviceaccount.com which is the active account specified by the [core/account] property.
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: iam.googleapis.com
  metadata:
    permission: iam.serviceAccounts.getIamPolicy
  reason: IAM_PERMISSION_DENIED


In [134]:
from __future__ import annotations

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from vertexai.vision_models import Image, Video, MultiModalEmbeddingModel

text_embedding_model="textembedding-gecko"
multimodal_embedding_model="multimodalembedding@002"

def embed_text(query) -> list[list[float]]:
    """Embeds texts with a pre-trained, foundational model.
    Args:
    str query: the query that should be converted to text embedding
    Returns:
        A list of lists containing the embedding vectors for each input text
    """
  
    # The task type for embedding. Check the available tasks in the model's documentation.
    task = "RETRIEVAL_DOCUMENT"

    model = TextEmbeddingModel.from_pretrained(text_embedding_model)
    inputs = [TextEmbeddingInput(query, task)]
    #kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
    embeddings = model.get_embeddings(inputs)
 
    # Example response:
    # [[0.006135190837085247, -0.01462465338408947, 0.004978656303137541, ...], [0.1234434666, ...]],
    return [embedding.values for embedding in embeddings]



def embed_multimodal(query,image,video) -> list[list[float]]:
    """Embeds texts with a pre-trained, foundational model.
    Args:
    str query: the query that should be converted to text embedding
    64bytestream image: the query in image format converted to 64bytestream or path -->talk to Mason
    64bytestream video: the query in video format converted to 64bytestream or path-->talk to Mason
    
    Returns:
        A list of lists containing the embedding vectors for each input text
    """
     
    model = MultiModalEmbeddingModel.from_pretrained(multimodal_embedding_model)
    #if image path given
    # image = Image.load_from_file(
    # "gs://cloud-samples-data/vertex-ai/llm/prompts/landmark1.png"
    #  )
    #if not, we have to convert the image to 64bytestream

    
     #if video path given
     # video = Video.load_from_file(
    # "gs://cloud-samples-data/vertex-ai/llm/prompts/landmark1.png"
    #  )
    #if not, we have to convert the video to 64bytestream
    
    if image: 
    embeddings = model.get_embeddings(
    image=image,
    #contextual_text="Colosseum"#,
    #dimension=embedding_dimension,
    )
    if video: 
    embeddings = model.get_embeddings(
    video=video,
    #contextual_text="Colosseum"#,
    #dimension=embedding_dimension,
    )
    if text: 
    embeddings = model.get_embeddings(
    contextual_text=query,
    #contextual_text="Colosseum"#,
    #dimension=embedding_dimension,
    )
  
    return embeddings.image_embedding,embeddings.text_embedding,embeddings.video_embedding

In [ ]:
def query_nearest_neighbors(query_embedding, table_name, top_k=50):
    """Query nearest neighbors using cosine similarity in BigQuery."""
 

    # SQL for finding the nearest neighbors using cosine similarity
    sql = f"""
    WITH query_embedding AS (
        SELECT {json.dumps(query_embedding)} AS embedding
    ),
    similarity_scores AS (
        SELECT 
            text,
            embedding,
            (SUM(a * b) / (SQRT(SUM(POW(a, 2))) * SQRT(SUM(POW(b, 2))))) AS cosine_similarity
        FROM
            `your_project-id.your_dataset.{table_name}`,
            UNNEST(embedding) AS a WITH OFFSET AS idx,
            UNNEST(query_embedding.embedding) AS b WITH OFFSET AS query_idx
        GROUP BY text, embedding
    )
    SELECT text, cosine_similarity
    FROM similarity_scores
    ORDER BY cosine_similarity DESC
    LIMIT {top_k}
    """

    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()

    return [(row.text, row.cosine_similarity) for row in results]